<a href="https://colab.research.google.com/github/IntrovertInDisguise/iitgn-robotics/blob/main/Vignesh_Ramakrishnan_Assignment6_7_ITR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Paisa Double bonus word: DEV

In [ ]:
#importing some useful libraries from miniproject
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import scipy.integrate as integrate 
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from matplotlib import rc
from IPython.display import HTML
from numpy import sin, cos,arctan,arccos
from collections import deque
import sympy as sp

!pip3 install roboticstoolbox-python
import roboticstoolbox as rtb
from spatialmath import SE3
import cv2
from mpl_toolkits import mplot3d
#from matplotlib import pyplot as plt
import spatialmath.base as base
#Acceleration due to gravity
g=9.81

     |████████████████████████████████| 497 kB 5.2 MB/s 
     |████████████████████████████████| 93.7 MB 1.2 MB/s 
     |████████████████████████████████| 5.7 MB 20.3 MB/s 
     |████████████████████████████████| 162 kB 41.1 MB/s 
     |████████████████████████████████| 111 kB 57.9 MB/s 
     |████████████████████████████████| 121 kB 51.8 MB/s 


In [ ]:
#Suraj Repo
robot1 = rtb.DHRobot(
        [
            rtb.RevoluteDH(alpha=0, a=0.5, d=0.2, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.RevoluteDH(alpha=np.pi, a=0.5, d=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.PrismaticDH(theta=0, alpha=0, a=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
        ], name="scararob", gravity=[0, 0, 9.81])

DH=[[0,0,0.2,0.5],[np.pi,0,0,0.5],[0,0,0,0]]


def scara_inverse_kinematics(DH,xc,yc,zc):
   d1=DH[0][2]
   a1 = DH[0][3]
   a2 = DH[1][3]
   D = (xc**2+yc**2-a1**2-a2**2 )/(2*a1*a2)
   theta2 = np.arctan2(np.sqrt(1-D**2),D)
   theta1 = np.arctan2(yc,xc)-np.arctan2(a2*np.sin(theta2),(a1+a2*np.cos(theta2)))
   d3 = d1-zc
   return theta1,theta2,d3
#print(scara_inverse_kinematics(DH,0.4,0.06,0.1))
def homogenous_matrix(DH1): # from textbook
    ct = np.cos(DH1[0])
    ca = np.cos(DH1[1])
    st = np.sin(DH1[0])
    sa = np.sin(DH1[1])
    a = DH1[1]
    d = DH1[2]
    r = DH1[3]
    homogenous_matrix =np.array([[ct, -st*ca,  st*sa,   r*ct],
                                [st,   ct*ca,  -ct*sa,  r*st],
                                [0,    sa,     ca,         d],
                                [0,    0,      0,          1]])
    return homogenous_matrix

def forward_kinematics(DH): # returns all link endpoints
    A = np.identity(4)
    n = np.shape(DH)[0]
    ox = [0]
    oy = [0]
    oz = [0]
    for i in range(n):
        Anext = homogenous_matrix(DH[i])
        A = np.matmul(A,Anext)

        ox.append(A[0][3])
        oy.append(A[1][3])
        oz.append(A[2][3])
    return ox,oy,oz
    #Test
# alpha1 = 0
# d1 = 0.2
# a1 = 0.5
# # link2
# alpha2 = np.deg2rad(180)
# d2 = 0
# a2 = 0.5
# # link3
# alpha3 = np.deg2rad(0)
# theta3 = 0
# a3 = 0
# d3 = 0
# DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]
# print(forward_kinematics(DH))
def forward_kinematics2(DH): # returns only endeffector
    A = np.identity(4)
    n = np.shape(DH)[0]
    ox = [0]
    oy = [0]
    oz = [0]
    for i in range(n):
        Anext = homogenous_matrix(DH[i])
        A = np.matmul(A,Anext)

    ox = (A[0][3])
    oy = (A[1][3])
    oz = (A[2][3])
    return ox,oy,oz

def plotrobotscara(DH): # plot in 3D 3 DOF robot from DH param. base link added

    ox,oy,oz = forward_kinematics(DH)

    link0x = [0,0]
    link0y = [0,0]
    link0z = [0,DH[0][2]]

    link1x = [ox[0],ox[1]]
    link1y = [oy[0],oy[1]]
    link1z = [oz[1],oz[1]]

    link2x = [ox[1],ox[2]]
    link2y = [oy[1],oy[2]]
    link2z = [oz[1],oz[2]]

    link3x = [ox[2],ox[3]]
    link3y = [oy[2],oy[3]]
    link3z = [oz[2],oz[3]]

    ax = plt.axes(projection='3d')

    ax.set_zlabel('z')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    elev = 45
    azim = 45
    ax.view_init(elev, azim)
    ax.set_xlim(-0.5, 0.5)
    ax.set_ylim(-0.5, 0.5)
    ax.set_zlim(0, 0.5)

    print('verification of endpoint by FK', ox[3], oy[3], oz[3])

    v = ax.plot3D(link0x, link0y, link0z, '-g',label='base', linewidth=2) #base link
    a = ax.plot3D(link1x, link1y, link1z, '-r',label='link1', linewidth=2)
    b = ax.plot3D(link2x, link2y, link2z, '-b',label='link2', linewidth=2)
    c = ax.plot3D(link3x, link3y, link3z, '-k',label='link3', linewidth=2)

    return ax



def cubic(x0, xf, x0dot, xfdot, t0, tf): # gives cubic polynomial coeff for single joint

    T = np.array([[1, t0, t0**2, t0**3],[0, 1, 2 * t0, 3 * t0**2],[1, tf, tf**2, tf**3],[0, 1, 2 * tf, 3 * tf**2]])
    x = np.array([[x0],[x0dot],[xf],[xfdot]])
    a = np.matmul(np.linalg.inv(T),x)
    return a


## Q1: Trajectory tracking



In [ ]:
A=[0.4,0.06,0.1]
B=[0.4,0.01,0.1]
t0=0
tf=10
t=np.linspace(t0,tf,num=100,endpoint=True)
#Constraints

#Displacement
x0=A[0]
xf=B[0]
y0=A[1]
yf=B[1]
z0=A[2]
zf=B[2]
xyz0=[x0,y0,z0]
xyzf=[xf,yf,zf]
#print(xyz0,"m Initial position")
#print(xyzf,"m Final position")
#xyzd=[i-j  for (i, j) in zip(xyzf, xyz0)]
#print(xyzd,"m Net displacement")

#Velocity
V0=[0,0,0]
Vf=[0,0,0]

#Finding cubic polynomial trajectory

Cube_poly=np.transpose(np.array([[1,0,1,0],[t0,1,tf,1],[t0**2,2*t0,tf**2,2*tf],[t0**3,3*(t0**2),tf**3,3*(tf**2)]]))
#print(Cube_poly)
#print(cubic(x0,V0[0],xf,V0[0],t0,tf))
# coeffs along x
ax= (np.linalg.inv(Cube_poly) @ np.array([x0,V0[0],xf,V0[0]]))
#print(ax)
# coeffs along x
ay=(np.linalg.inv(Cube_poly) @ np.array([y0,V0[1],yf,V0[1]]))
#print(ay)
# coeffs along z
az= (np.linalg.inv(Cube_poly) @ np.array([z0,V0[2],zf,V0[2]]))
#print(az)

#Displacement
tpolyx=np.array(ax)@np.transpose(np.array([1,t,t**2,t**3]))
tpolyy=np.array(ay)@np.transpose(np.array([1,t,t**2,t**3]))
tpolyz=np.array(az)@np.transpose(np.array([1,t,t**2,t**3]))
#Velo
tpolyvx=np.array(ax[1:])@np.transpose(np.array([1,2*t,3*t**2]))
tpolyvy=np.array(ay[1:])@np.transpose(np.array([1,2*t,3*t**2]))
tpolyvz=np.array(az[1:])@np.transpose(np.array([1,2*t,3*t**2]))
#Acceleration
tpolyax=np.array(ax[2:])@np.transpose(np.array([2,6*t]))
tpolyay=np.array(ay[2:])@np.transpose(np.array([2,6*t]))
tpolyaz=np.array(az[2:])@np.transpose(np.array([2,6*t]))



fig = plt.figure(0)

ax = fig.add_subplot(321)
ax.set_title("Displacement in m")
ax.set_xlabel("time in s")
ax.set_ylabel("x")
ax.plot(t,np.transpose(tpolyx))


ax = fig.add_subplot(323)
ax.set_title("Displacement")
ax.plot(t,np.transpose(tpolyy))
ax.set_xlabel("time in s")
ax.set_ylabel("y")


ax = fig.add_subplot(325)
ax.set_title("Displacement")
ax.plot(t,np.transpose(tpolyz))
ax.set_xlabel("time in s")
ax.set_ylabel("z")

fig1 = plt.figure(1)
ax = fig1.add_subplot(321)
ax.set_title("Velocity in m/s")
ax.set_xlabel("time in s")
ax.set_ylabel("Vx")
ax.plot(t,np.transpose(tpolyvx))


ax = fig1.add_subplot(323)
ax.set_title("Velocity")
ax.plot(t,np.transpose(tpolyvy))
ax.set_xlabel("time in s")
ax.set_ylabel("Vy")


ax = fig1.add_subplot(325)
ax.set_title("Velocity")
ax.plot(t,np.transpose(tpolyvz))
ax.set_xlabel("time in s")
ax.set_ylabel("Vz")

fig2 = plt.figure(2)
ax = fig2.add_subplot(321)
ax.set_title("Acceleration in m/s2")
ax.set_xlabel("time in s")
ax.set_ylabel("Ax")
ax.plot(t,np.transpose(tpolyax))


ax = fig2.add_subplot(323)
ax.set_title("Acceleration")
ax.plot(t,np.transpose(tpolyay))
ax.set_xlabel("time in s")
ax.set_ylabel("Ay")


ax = fig2.add_subplot(325)
ax.set_title("Acceleration")
ax.plot(t,np.transpose(tpolyaz))
ax.set_xlabel("time in s")
ax.set_ylabel("Az")



##Q2: Joint Space Tracking for SCARA manipulator

In [ ]:
#Desired Joint trajectories
theta1poly,theta2poly,dzpoly=[0]*len(t),[0]*len(t),[0]*len(t)
for i in range(0,len(t)):
  theta1poly[i],theta2poly[i],dzpoly[i]=scara_inverse_kinematics(DH,tpolyx[i],tpolyy[i],tpolyz[i])
fig3 = plt.figure(3)
ax = fig3.add_subplot(321)
ax.set_title("Angular displacement")
ax.set_xlabel("time in s")
ax.set_ylabel("theta 1")
ax.plot(t,np.transpose(theta1poly))


ax = fig3.add_subplot(323)
ax.set_title("Angular displacement")
ax.set_xlabel("time in s")
ax.set_ylabel("theta 2")
ax.plot(t,np.transpose(theta2poly))



ax = fig3.add_subplot(325)
ax.set_title("Linear displacement")
ax.plot(t,np.transpose(dzpoly))
ax.set_xlabel("time in s")
ax.set_ylabel("dz")

##Q3 a) PD control with $\zeta=1$

In [ ]:
#Suraj Repo Dynamics + control
def modelscara(q, t, i, I1, I2, l1, l2, l3, lc1, lc2, lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2, r3, R1, R2, R3, Km1, Km2, Km3, Kb1, Kb2, Kb3,
               Bm1, Bm2, Bm3, zeta, wn, K1, K2, K3,q1_poly,q2_poly,q3_poly):

    dt = 0.01 # step size change accordingly
    t1 = q[0]
    t2 = q[1]
    t3 = q[2]
    #joint rates
    dt1 = q[3]
    dt2 = q[4]
    dt3 = q[5]

    # cubic desired trajectory
    qd1 = q1_poly[0]+q1_poly[1]*t+q1_poly[2]*t**2+q1_poly[3]*t**3
    qd2 = q2_poly[0] + q2_poly[1] * t + q2_poly[2] * t ** 2 + q2_poly[3] * t ** 3
    qd3 = q3_poly[0] + q3_poly[1] * t + q3_poly[2] * t ** 2 + q3_poly[3] * t ** 3
    # desired velocity
    dqd1 = q1_poly[1]  + 2*q1_poly[2] * t+ 3*q1_poly[3] * t **2
    dqd2 = q2_poly[1]  + 2*q2_poly[2] * t+ 3*q2_poly[3] * t **2
    dqd3 = q3_poly[1]  + 2*q3_poly[2] * t + 3*q3_poly[3] * t **2
    # desired acceleration
    ddqd1 =  2*q1_poly[2]+ 6*q1_poly[3] * t
    ddqd2 =  2*q2_poly[2]+ 6*q2_poly[3] * t
    ddqd3 =  2*q3_poly[2] + 6*q3_poly[3] * t

    # desired accl vector
    tdd = np.array([[ddqd1],[ddqd2],[ddqd2]])

    tdd = np.reshape(tdd,(3,1))
    # actual velocity vector
    td = np.array([[dt1],[dt2],[dt3]])

    tau = np.transpose(np.zeros(3))
    # Define Robot using PeterCorke toolbox just substitute non variable DH parameters Inertia and mass
    robot1 = rtb.DHRobot(
        [
            rtb.RevoluteDH(alpha=0, a=0.5, d=0.2, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.RevoluteDH(alpha=np.pi, a=0.5, d=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.PrismaticDH(theta=0, alpha=0, a=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
        ], name="scararob", gravity=[0, 0, 9.81])

    # Get D,C,phi
    D = robot1.inertia([t1,t2,t3])
    C = robot1.coriolis([t1,t2,t3],[dt1,dt2,dt3])
    phi = robot1.gravload([t1,t2,t3])
    phi = np.reshape(phi, (3, 1))

    #diagonal terms D to dkk
    dkk = np.array([[D[0][0], 0, 0],[0, D[1][1],0],[0, 0,D[2][2]]])
    # modelling disturbance, you can more disturbances here
    dk =   np.matmul(C, td) +np.matmul((D-dkk), tdd) +phi
    #motor parameters
    Jm = np.array([[Jm1,0,0],[0,Jm2,0],[0,0,Jm3]])
    Jeff1 = Jm1 + r1 * r1 * dkk[0][0]
    Jeff2 = Jm2 + r2 * r2 * dkk[1][1]
    Jeff3 = Jm3 + r3 * r3 * dkk[2][2]
    Beff1 = Bm1 + Kb1 * Km1 / R1
    Beff2 = Bm2 + Kb2 * Km2 / R2
    Beff3 = Bm3 + Kb3 * Km3 / R2
    K1 = Km1 / R1
    K2 = Km2 / R2
    K3 = Km3 / R3
    #gear ratio will assume same gear ratio so will only use scalar r1
    r = np.array([[r1, 0, 0],[0, r2,0],[0,0,r3]])
    Jeff = np.array([[Jeff1, 0,0],[0, Jeff2,0],[0,0,Jeff3]])
    Jeffinv = np.linalg.inv(Jeff)
    Beff = np.array([[Beff1, 0,0],[0, Beff2,0],[0,0,Beff3]])

    K = np.array([[K1,0,0],[0,K2,0],[0,0,K3]])/dt

    Kinv = np.linalg.inv(K)

    # compute errors
    e = np.array([[qd1-q[0]],[qd2-q[1]],[qd3-q[2]]])
    e = np.reshape(e,(3,1))
    ed = np.array([[0 - q[3]], [0 - q[4]], [0 - q[5]]])
    ed = np.reshape(ed, (3, 1))
    
    #PID Gains
    Kp = wn * wn * np.matmul(Jeff,  Kinv)
    
    Kd = np.matmul((2 * zeta * wn * Jeff - Beff ),Kinv) # change zeta here

    dt = 0.01
    # if i == 0:
    #     tprev = 0
    #     dt = t - tprev
    # else:
    #     dt = t - tprev
    #disp(size(dt));
    #sum = np.matmul(Ki, e) * dt

    V = np.matmul(Kp , e) + np.matmul(Kd ,ed) # voltage control input

    # dynamic equation in joint varaibles
    ddq = np.matmul(np.linalg.inv(Jeff),(r1*np.matmul(K ,V) - (r1**2)*(dk)- np.matmul(Beff , td)))
    # find statedot
    dq = [q[3],q[4],q[5],ddq[0][0],ddq[1][0],ddq[2][0]]

    #print('t= '+str(t))
    # tprev = t used for Integral
    # i = i+1
    return dq


In [ ]:
#Suraj code main
#####################################################################################################################
## DEFINE DH PARAMETERS

# link1
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]
# plot scara
ax = plotrobotscara(DH) # to see plot need to do plt.show()
plt.show()
## OBTAIN JOINT TRAJECTORY USING END EFFECTOR TRAJECTORY
q0 = scara_inverse_kinematics(DH,0.4,0.01,0.1)
q1 = scara_inverse_kinematics(DH,0.4,0.06,0.1)
q1_poly = cubic(q0[0], q1[0], 0, 0, 0, 1)
q2_poly = cubic(q0[1], q1[1], 0, 0, 0, 1)
q3_poly = cubic(q0[2], q1[2], 0, 0, 0, 1)

# time from (t0,tfinal,no)
t=np.linspace(0,1,100)

q = np.transpose(np.zeros(6))
i = 0
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]  # sequence [theta,alpha,d,a]
# initial configuration
qi = scara_inverse_kinematics(DH,0.4,0.01,0.1)
#start state for integrator [q,dq] last three values indicate joint rate
q0 = [qi[0],qi[1],qi[2],0,0,0]
# motor and link parameter values
m1=1 ; m2=1 ; l1=1;l2=1; l3=1; I1=1/12; I2=1/12; I3=1/12; lc1=l1/2; lc2=l2/2; lc3=l3/2; g=9.81
Jm1 =0.4*10**-4; Jm2 =0.4*10**-4; Jm3=0.4*10**(-4); Bm1 = 4*10**(-5); Bm2 = 4*10**-5; Bm3=4*10**-5; Km1 =2*10**-2; Km2 =2*10**-2;Km3 =2*10**-2
Kb1 = 0.02;Kb2 = 0.02;Kb3=0.02;R1 = 0.3;R2 = 0.3;R3 = 0.3;r1 = 100;r2 = 100; r3=100; zeta = 1; wn = 50; K1 = 1; K2 = 1; K3 = 1

i = 0
### integrate the robot dynamics and control
q = integrate.odeint(modelscara,q0,t,args=(i, I1, I2, l1, l2,l3, lc1, lc2,lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2,r3, R1, R2,R3, Km1, Km2,Km3, Kb1, Kb2, Kb3,Bm1, Bm2, Bm3, zeta, wn,K1,K2,K3,q1_poly,q2_poly,q3_poly))

print("q",q)
xed = 0.4 * np.ones(100) # end-effector path only for plotting purpose
yed = np.linspace(0.01, 0.06, 100)
zed = 0.1 * np.ones(100)

DH_curr=[[q[:,0], alpha1, d1, a1], [q[:,1], alpha2, d2, a2], [theta3, alpha3, q[:,2], a3]]
xe,ye,ze=forward_kinematics2(DH_curr)


plt.figure(1)
tt = np.linspace(0,1,100)


# generate actual vs desired plots here

f=plt.figure(2)
for i in range(len(tt)):
  ax=f.add_subplot(111)
  ax.plot(tt,xed,color='r',label='Desired trajectory x')
  ax.plot(tt,xe,color='g',label='Actual trajectory x')

f1=plt.figure(3)
for i in range(len(tt)):
  ax=f1.add_subplot(111)
  ax.plot(tt,yed,color='r',label='Desired trajectory y')
  ax.plot(tt,ye,color='g',label='Actual trajectory y')
  
# Animate in 3D
for i in range(len(tt)):
    # substitute joint variables obtained from integration
    DH = [[q[i][0], alpha1, d1, a1], [q[i][1], alpha2, d2, a2], [theta3, alpha3, q[i][2], a3]]
    ax = plotrobotscara(DH)
    ax.plot3D(xed, yed, zed, '-m', label=r'$desired end-effector trajectory$')
    plt.draw()
    ax.text(0.5, 0.5, 0, 'step no ='+str(i), color='red')
    plt.pause(0.001)



##Q3 b) Independent joint control i.e. PD+Feedforward using motor dynamics


In [ ]:
#Suraj Repo Dynamics + control
def modelscara(q, t, i, I1, I2, l1, l2, l3, lc1, lc2, lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2, r3, R1, R2, R3, Km1, Km2, Km3, Kb1, Kb2, Kb3,
               Bm1, Bm2, Bm3, zeta, wn, K1, K2, K3,q1_poly,q2_poly,q3_poly):

    dt = 0.01 # step size change accordingly
    t1 = q[0]
    t2 = q[1]
    t3 = q[2]
    #joint rates
    dt1 = q[3]
    dt2 = q[4]
    dt3 = q[5]

    # cubic desired trajectory
    qd1 = q1_poly[0]+q1_poly[1]*t+q1_poly[2]*t**2+q1_poly[3]*t**3
    qd2 = q2_poly[0] + q2_poly[1] * t + q2_poly[2] * t ** 2 + q2_poly[3] * t ** 3
    qd3 = q3_poly[0] + q3_poly[1] * t + q3_poly[2] * t ** 2 + q3_poly[3] * t ** 3
    # desired velocity
    dqd1 = q1_poly[1]  + 2*q1_poly[2] * t+ 3*q1_poly[3] * t **2
    dqd2 = q2_poly[1]  + 2*q2_poly[2] * t+ 3*q2_poly[3] * t **2
    dqd3 = q3_poly[1]  + 2*q3_poly[2] * t + 3*q3_poly[3] * t **2
    # desired acceleration
    ddqd1 =  2*q1_poly[2]+ 6*q1_poly[3] * t
    ddqd2 =  2*q2_poly[2]+ 6*q2_poly[3] * t
    ddqd3 =  2*q3_poly[2] + 6*q3_poly[3] * t

    # desired accl vector
    tdd = np.array([[ddqd1],[ddqd2],[ddqd2]])

    tdd = np.reshape(tdd,(3,1))
    # actual velocity vector
    td = np.array([[dt1],[dt2],[dt3]])

    tau = np.transpose(np.zeros(3))
    # Define Robot using PeterCorke toolbox just substitute non variable DH parameters Inertia and mass
    robot1 = rtb.DHRobot(
        [
            rtb.RevoluteDH(alpha=0, a=0.5, d=0.2, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.RevoluteDH(alpha=np.pi, a=0.5, d=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.PrismaticDH(theta=0, alpha=0, a=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
        ], name="scararob", gravity=[0, 0, 9.81])

    # Get D,C,phi
    D = robot1.inertia([t1,t2,t3])
    C = robot1.coriolis([t1,t2,t3],[dt1,dt2,dt3])
    phi = robot1.gravload([t1,t2,t3])
    phi = np.reshape(phi, (3, 1))

    #diagonal terms D to dkk
    dkk = np.array([[D[0][0], 0, 0],[0, D[1][1],0],[0, 0,D[2][2]]])
    # modelling disturbance, you can more disturbances here
    dk =   np.matmul(C, td) +np.matmul((D-dkk), tdd) +phi
    #motor parameters
    Jm = np.array([[Jm1,0,0],[0,Jm2,0],[0,0,Jm3]])
    Jeff1 = Jm1 + r1 * r1 * dkk[0][0]
    Jeff2 = Jm2 + r2 * r2 * dkk[1][1]
    Jeff3 = Jm3 + r3 * r3 * dkk[2][2]
    Beff1 = Bm1 + Kb1 * Km1 / R1
    Beff2 = Bm2 + Kb2 * Km2 / R2
    Beff3 = Bm3 + Kb3 * Km3 / R2
    K1 = Km1 / R1
    K2 = Km2 / R2
    K3 = Km3 / R3
    #gear ratio will assume same gear ratio so will only use scalar r1
    r = np.array([[r1, 0, 0],[0, r2,0],[0,0,r3]])
    Jeff = np.array([[Jeff1, 0,0],[0, Jeff2,0],[0,0,Jeff3]])
    Jeffinv = np.linalg.inv(Jeff)
    Beff = np.array([[Beff1, 0,0],[0, Beff2,0],[0,0,Beff3]])

    K = np.array([[K1,0,0],[0,K2,0],[0,0,K3]])/dt

    Kinv = np.linalg.inv(K)

    # compute errors
    e = np.array([[qd1-q[0]],[qd2-q[1]],[qd3-q[2]]])
    e = np.reshape(e,(3,1))
    ed = np.array([[0 - q[3]], [0 - q[4]], [0 - q[5]]])
    ed = np.reshape(e, (3, 1))
    
    #PID Gains
    f=np.matmul(np.matmul(Jeff,  Kinv),[ddqd1,ddqd2,ddqd3])+np.matmul(np.matmul(Beff,  Kinv),[dqd1,dqd2,dqd3])
    print(f)

    Kp = wn * wn * np.matmul(Jeff,  Kinv)
    
    Kd = np.matmul((2 * zeta * wn * Jeff - Beff ),Kinv) # change zeta here

    dt = 0.01
    # if i == 0:
    #     tprev = 0
    #     dt = t - tprev
    # else:
    #     dt = t - tprev
    #disp(size(dt));
    #sum = np.matmul(Ki, e) * dt

    V = np.matmul(Kp , e) + np.matmul(Kd ,ed) + f # voltage control input

    # dynamic equation in joint varaibles
    ddq = np.matmul(np.linalg.inv(Jeff),(r1*np.matmul(K ,V) - (r1**2)*(dk)- np.matmul(Beff , td)))
    # find statedot
    dq = [q[3],q[4],q[5],ddq[0][0],ddq[1][0],ddq[2][0]]

    #print('t= '+str(t))
    # tprev = t used for Integral
    # i = i+1
    return dq


In [ ]:
#Suraj code main
#####################################################################################################################
## DEFINE DH PARAMETERS

# link1
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]
# plot scara
ax = plotrobotscara(DH) # to see plot need to do plt.show()
plt.show()
## OBTAIN JOINT TRAJECTORY USING END EFFECTOR TRAJECTORY
q0 = scara_inverse_kinematics(DH,0.4,0.01,0.1)
q1 = scara_inverse_kinematics(DH,0.4,0.06,0.1)
q1_poly = cubic(q0[0], q1[0], 0, 0, 0, 1)
q2_poly = cubic(q0[1], q1[1], 0, 0, 0, 1)
q3_poly = cubic(q0[2], q1[2], 0, 0, 0, 1)

# time from (t0,tfinal,no)
t=np.linspace(0,1,100)

q = np.transpose(np.zeros(6))
i = 0
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]  # sequence [theta,alpha,d,a]
# initial configuration
qi = scara_inverse_kinematics(DH,0.4,0.01,0.1)
#start state for integrator [q,dq] last three values indicate joint rate
q0 = [qi[0],qi[1],qi[2],0,0,0]
# motor and link parameter values
m1=1 ; m2=1 ; l1=1;l2=1; l3=1; I1=1/12; I2=1/12; I3=1/12; lc1=l1/2; lc2=l2/2; lc3=l3/2; g=9.81
Jm1 =0.4*10**-4; Jm2 =0.4*10**-4; Jm3=0.4*10**(-4); Bm1 = 4*10**(-5); Bm2 = 4*10**-5; Bm3=4*10**-5; Km1 =2*10**-2; Km2 =2*10**-2;Km3 =2*10**-2
Kb1 = 0.02;Kb2 = 0.02;Kb3=0.02;R1 = 0.3;R2 = 0.3;R3 = 0.3;r1 = 100;r2 = 100; r3=100; zeta = 1; wn = 50; K1 = 1; K2 = 1; K3 = 1

i = 0
### integrate the robot dynamics and control
q = integrate.odeint(modelscara,q0,t,args=(i, I1, I2, l1, l2,l3, lc1, lc2,lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2,r3, R1, R2,R3, Km1, Km2,Km3, Kb1, Kb2, Kb3,Bm1, Bm2, Bm3, zeta, wn,K1,K2,K3,q1_poly,q2_poly,q3_poly))


xed = 0.4 * np.ones(100) # end-effector path only for plotting purpose
yed = np.linspace(0.01, 0.06, 100)
zed = 0.1 * np.ones(100)

DH_curr=[[q[:,0], alpha1, d1, a1], [q[:,1], alpha2, d2, a2], [theta3, alpha3, q[:,2], a3]]
xe,ye,ze=forward_kinematics2(DH_curr)


plt.figure(1)
tt = np.linspace(0,1,100)


# generate actual vs desired plots here

f=plt.figure(2)
for i in range(len(tt)):
  ax=f.add_subplot(111)
  ax.plot(tt,xed,color='r',label='Desired trajectory x')
  ax.plot(tt,xe,color='g',label='Actual trajectory x')

f1=plt.figure(3)
for i in range(len(tt)):
  ax=f1.add_subplot(111)
  ax.plot(tt,yed,color='r',label='Desired trajectory y')
  ax.plot(tt,ye,color='g',label='Actual trajectory y')
  
# Animate in 3D
for i in range(len(tt)):
    # substitute joint variables obtained from integration
    DH = [[q[i][0], alpha1, d1, a1], [q[i][1], alpha2, d2, a2], [theta3, alpha3, q[i][2], a3]]
    ax = plotrobotscara(DH)
    ax.plot3D(xed, yed, zed, '-m', label=r'$desired end-effector trajectory$')
    plt.draw()
    ax.text(0.5, 0.5, 0, 'step no ='+str(i), color='red')
    plt.pause(0.001)



##Q3 c) Independent joint control i.e. PD+Feedforward using motor dynamics with computed torque


In [ ]:
#Suraj Repo Dynamics + control
def modelscara(q, t, i, I1, I2, l1, l2, l3, lc1, lc2, lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2, r3, R1, R2, R3, Km1, Km2, Km3, Kb1, Kb2, Kb3,
               Bm1, Bm2, Bm3, zeta, wn, K1, K2, K3,q1_poly,q2_poly,q3_poly):

    dt = 0.01 # step size change accordingly
    t1 = q[0]
    t2 = q[1]
    t3 = q[2]
    #joint rates
    dt1 = q[3]
    dt2 = q[4]
    dt3 = q[5]

    # cubic desired trajectory
    qd1 = q1_poly[0]+q1_poly[1]*t+q1_poly[2]*t**2+q1_poly[3]*t**3
    qd2 = q2_poly[0] + q2_poly[1] * t + q2_poly[2] * t ** 2 + q2_poly[3] * t ** 3
    qd3 = q3_poly[0] + q3_poly[1] * t + q3_poly[2] * t ** 2 + q3_poly[3] * t ** 3
    # desired velocity
    dqd1 = q1_poly[1]  + 2*q1_poly[2] * t+ 3*q1_poly[3] * t **2
    dqd2 = q2_poly[1]  + 2*q2_poly[2] * t+ 3*q2_poly[3] * t **2
    dqd3 = q3_poly[1]  + 2*q3_poly[2] * t + 3*q3_poly[3] * t **2
    # desired acceleration
    ddqd1 =  2*q1_poly[2]+ 6*q1_poly[3] * t
    ddqd2 =  2*q2_poly[2]+ 6*q2_poly[3] * t
    ddqd3 =  2*q3_poly[2] + 6*q3_poly[3] * t

    # desired accl vector
    tdd = np.array([[ddqd1],[ddqd2],[ddqd2]])

    tdd = np.reshape(tdd,(3,1))
    # actual velocity vector
    td = np.array([[dt1],[dt2],[dt3]])

    tau = np.transpose(np.zeros(3))
    # Define Robot using PeterCorke toolbox just substitute non variable DH parameters Inertia and mass
    robot1 = rtb.DHRobot(
        [
            rtb.RevoluteDH(alpha=0, a=0.5, d=0.2, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.RevoluteDH(alpha=np.pi, a=0.5, d=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.PrismaticDH(theta=0, alpha=0, a=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
        ], name="scararob", gravity=[0, 0, 9.81])

    # Get D,C,phi
    D = robot1.inertia([t1,t2,t3])
    C = robot1.coriolis([t1,t2,t3],[dt1,dt2,dt3])
    phi = robot1.gravload([t1,t2,t3])
    phi = np.reshape(phi, (3, 1))

    #diagonal terms D to dkk
    dkk = np.array([[D[0][0], 0, 0],[0, D[1][1],0],[0, 0,D[2][2]]])
    # modelling disturbance, you can more disturbances here
    dk =   np.matmul(C, td) +np.matmul((D-dkk), tdd) +phi
    #motor parameters
    Jm = np.array([[Jm1,0,0],[0,Jm2,0],[0,0,Jm3]])
    Jeff1 = Jm1 + r1 * r1 * dkk[0][0]
    Jeff2 = Jm2 + r2 * r2 * dkk[1][1]
    Jeff3 = Jm3 + r3 * r3 * dkk[2][2]
    Beff1 = Bm1 + Kb1 * Km1 / R1
    Beff2 = Bm2 + Kb2 * Km2 / R2
    Beff3 = Bm3 + Kb3 * Km3 / R2
    K1 = Km1 / R1
    K2 = Km2 / R2
    K3 = Km3 / R3
    #gear ratio will assume same gear ratio so will only use scalar r1
    r = np.array([[r1, 0, 0],[0, r2,0],[0,0,r3]])
    Jeff = np.array([[Jeff1, 0,0],[0, Jeff2,0],[0,0,Jeff3]])
    Jeffinv = np.linalg.inv(Jeff)
    Beff = np.array([[Beff1, 0,0],[0, Beff2,0],[0,0,Beff3]])

    K = np.array([[K1,0,0],[0,K2,0],[0,0,K3]])/dt

    Kinv = np.linalg.inv(K)

    # compute errors
    e = np.array([[qd1-q[0]],[qd2-q[1]],[qd3-q[2]]])
    e = np.reshape(e,(3,1))
    ed = np.array([[0 - q[3]], [0 - q[4]], [0 - q[5]]])
    ed = np.reshape(e, (3, 1))
    
    #PID Gains
    f=np.matmul(np.matmul(Jeff,  Kinv),[ddqd1,ddqd2,ddqd3])+np.matmul(np.matmul(Beff,  Kinv),[dqd1,dqd2,dqd3])
    print(f)

    Kp = wn * wn * np.matmul(Jeff,  Kinv)
    
    Kd = np.matmul((2 * zeta * wn * Jeff - Beff ),Kinv) # change zeta here
    tau=dk

    dt = 0.01
    # if i == 0:
    #     tprev = 0
    #     dt = t - tprev
    # else:
    #     dt = t - tprev
    #disp(size(dt));
    #sum = np.matmul(Ki, e) * dt

    V = np.matmul(Kp , e) + np.matmul(Kd ,ed) + f +tau# voltage control input

    # dynamic equation in joint varaibles
    ddq = np.matmul(np.linalg.inv(Jeff),(r1*np.matmul(K ,V) - (r1**2)*(dk)- np.matmul(Beff , td)))
    # find statedot
    dq = [q[3],q[4],q[5],ddq[0][0],ddq[1][0],ddq[2][0]]

    #print('t= '+str(t))
    # tprev = t used for Integral
    # i = i+1
    return dq


In [ ]:
#Suraj code main
#####################################################################################################################
## DEFINE DH PARAMETERS

# link1
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]
# plot scara
ax = plotrobotscara(DH) # to see plot need to do plt.show()
plt.show()
## OBTAIN JOINT TRAJECTORY USING END EFFECTOR TRAJECTORY
q0 = scara_inverse_kinematics(DH,0.4,0.01,0.1)
q1 = scara_inverse_kinematics(DH,0.4,0.06,0.1)
q1_poly = cubic(q0[0], q1[0], 0, 0, 0, 1)
q2_poly = cubic(q0[1], q1[1], 0, 0, 0, 1)
q3_poly = cubic(q0[2], q1[2], 0, 0, 0, 1)

# time from (t0,tfinal,no)
t=np.linspace(0,1,100)

q = np.transpose(np.zeros(6))
i = 0
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]  # sequence [theta,alpha,d,a]
# initial configuration
qi = scara_inverse_kinematics(DH,0.4,0.01,0.1)
#start state for integrator [q,dq] last three values indicate joint rate
q0 = [qi[0],qi[1],qi[2],0,0,0]
# motor and link parameter values
m1=1 ; m2=1 ; l1=1;l2=1; l3=1; I1=1/12; I2=1/12; I3=1/12; lc1=l1/2; lc2=l2/2; lc3=l3/2; g=9.81
Jm1 =0.4*10**-4; Jm2 =0.4*10**-4; Jm3=0.4*10**(-4); Bm1 = 4*10**(-5); Bm2 = 4*10**-5; Bm3=4*10**-5; Km1 =2*10**-2; Km2 =2*10**-2;Km3 =2*10**-2
Kb1 = 0.02;Kb2 = 0.02;Kb3=0.02;R1 = 0.3;R2 = 0.3;R3 = 0.3;r1 = 100;r2 = 100; r3=100; zeta = 1; wn = 50; K1 = 1; K2 = 1; K3 = 1

i = 0
### integrate the robot dynamics and control
q = integrate.odeint(modelscara,q0,t,args=(i, I1, I2, l1, l2,l3, lc1, lc2,lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2,r3, R1, R2,R3, Km1, Km2,Km3, Kb1, Kb2, Kb3,Bm1, Bm2, Bm3, zeta, wn,K1,K2,K3,q1_poly,q2_poly,q3_poly))


xed = 0.4 * np.ones(100) # end-effector path only for plotting purpose
yed = np.linspace(0.01, 0.06, 100)
zed = 0.1 * np.ones(100)

DH_curr=[[q[:,0], alpha1, d1, a1], [q[:,1], alpha2, d2, a2], [theta3, alpha3, q[:,2], a3]]
xe,ye,ze=forward_kinematics2(DH_curr)


plt.figure(1)
tt = np.linspace(0,1,100)


# generate actual vs desired plots here

f=plt.figure(2)
for i in range(len(tt)):
  ax=f.add_subplot(111)
  ax.plot(tt,xed,color='r',label='Desired trajectory x')
  ax.plot(tt,xe,color='g',label='Actual trajectory x')

f1=plt.figure(3)
for i in range(len(tt)):
  ax=f1.add_subplot(111)
  ax.plot(tt,yed,color='r',label='Desired trajectory y')
  ax.plot(tt,ye,color='g',label='Actual trajectory y')
  
# Animate in 3D
for i in range(len(tt)):
    # substitute joint variables obtained from integration
    DH = [[q[i][0], alpha1, d1, a1], [q[i][1], alpha2, d2, a2], [theta3, alpha3, q[i][2], a3]]
    ax = plotrobotscara(DH)
    ax.plot3D(xed, yed, zed, '-m', label=r'$desired end-effector trajectory$')
    plt.draw()
    ax.text(0.5, 0.5, 0, 'step no ='+str(i), color='red')
    plt.pause(0.001)



##Q4 a) PD control with $\zeta=1$ with 20% error in length

In [ ]:
#Suraj Repo Dynamics + control
def modelscara(q, t, i, I1, I2, l1, l2, l3, lc1, lc2, lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2, r3, R1, R2, R3, Km1, Km2, Km3, Kb1, Kb2, Kb3,
               Bm1, Bm2, Bm3, zeta, wn, K1, K2, K3,q1_poly,q2_poly,q3_poly):

    dt = 0.01 # step size change accordingly
    t1 = q[0]
    t2 = q[1]
    t3 = q[2]
    #joint rates
    dt1 = q[3]
    dt2 = q[4]
    dt3 = q[5]

    # cubic desired trajectory
    qd1 = q1_poly[0]+q1_poly[1]*t+q1_poly[2]*t**2+q1_poly[3]*t**3
    qd2 = q2_poly[0] + q2_poly[1] * t + q2_poly[2] * t ** 2 + q2_poly[3] * t ** 3
    qd3 = q3_poly[0] + q3_poly[1] * t + q3_poly[2] * t ** 2 + q3_poly[3] * t ** 3
    # desired velocity
    dqd1 = q1_poly[1]  + 2*q1_poly[2] * t+ 3*q1_poly[3] * t **2
    dqd2 = q2_poly[1]  + 2*q2_poly[2] * t+ 3*q2_poly[3] * t **2
    dqd3 = q3_poly[1]  + 2*q3_poly[2] * t + 3*q3_poly[3] * t **2
    # desired acceleration
    ddqd1 =  2*q1_poly[2]+ 6*q1_poly[3] * t
    ddqd2 =  2*q2_poly[2]+ 6*q2_poly[3] * t
    ddqd3 =  2*q3_poly[2] + 6*q3_poly[3] * t

    # desired accl vector
    tdd = np.array([[ddqd1],[ddqd2],[ddqd2]])

    tdd = np.reshape(tdd,(3,1))
    # actual velocity vector
    td = np.array([[dt1],[dt2],[dt3]])

    tau = np.transpose(np.zeros(3))
    # Define Robot using PeterCorke toolbox just substitute non variable DH parameters Inertia and mass
    robot1 = rtb.DHRobot(
        [
            rtb.RevoluteDH(alpha=0, a=0.5, d=0.2, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.RevoluteDH(alpha=np.pi, a=0.5, d=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.PrismaticDH(theta=0, alpha=0, a=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
        ], name="scararob", gravity=[0, 0, 9.81])

    # Get D,C,phi
    D = robot1.inertia([t1,t2,t3])
    C = robot1.coriolis([t1,t2,t3],[dt1,dt2,dt3])
    phi = robot1.gravload([t1,t2,t3])
    phi = np.reshape(phi, (3, 1))

    #diagonal terms D to dkk
    dkk = np.array([[D[0][0], 0, 0],[0, D[1][1],0],[0, 0,D[2][2]]])
    # modelling disturbance, you can more disturbances here
    dk =   np.matmul(C, td) +np.matmul((D-dkk), tdd) +phi
    #motor parameters
    Jm = np.array([[Jm1,0,0],[0,Jm2,0],[0,0,Jm3]])
    Jeff1 = Jm1 + r1 * r1 * dkk[0][0]
    Jeff2 = Jm2 + r2 * r2 * dkk[1][1]
    Jeff3 = Jm3 + r3 * r3 * dkk[2][2]
    Beff1 = Bm1 + Kb1 * Km1 / R1
    Beff2 = Bm2 + Kb2 * Km2 / R2
    Beff3 = Bm3 + Kb3 * Km3 / R2
    K1 = Km1 / R1
    K2 = Km2 / R2
    K3 = Km3 / R3
    #gear ratio will assume same gear ratio so will only use scalar r1
    r = np.array([[r1, 0, 0],[0, r2,0],[0,0,r3]])
    Jeff = np.array([[Jeff1, 0,0],[0, Jeff2,0],[0,0,Jeff3]])
    Jeffinv = np.linalg.inv(Jeff)
    Beff = np.array([[Beff1, 0,0],[0, Beff2,0],[0,0,Beff3]])

    K = np.array([[K1,0,0],[0,K2,0],[0,0,K3]])/dt

    Kinv = np.linalg.inv(K)

    # compute errors
    e = np.array([[qd1-q[0]],[qd2-q[1]],[qd3-q[2]]])
    e = np.reshape(e,(3,1))
    ed = np.array([[0 - q[3]], [0 - q[4]], [0 - q[5]]])
    ed = np.reshape(ed, (3, 1))
    
    #PID Gains
    Kp = wn * wn * np.matmul(Jeff,  Kinv)
    
    Kd = np.matmul((2 * zeta * wn * Jeff - Beff ),Kinv) # change zeta here

    dt = 0.01
    # if i == 0:
    #     tprev = 0
    #     dt = t - tprev
    # else:
    #     dt = t - tprev
    #disp(size(dt));
    #sum = np.matmul(Ki, e) * dt

    V = np.matmul(Kp , e) + np.matmul(Kd ,ed) # voltage control input

    # dynamic equation in joint varaibles
    ddq = np.matmul(np.linalg.inv(Jeff),(r1*np.matmul(K ,V) - (r1**2)*(dk)- np.matmul(Beff , td)))
    # find statedot
    dq = [q[3],q[4],q[5],ddq[0][0],ddq[1][0],ddq[2][0]]

    #print('t= '+str(t))
    # tprev = t used for Integral
    # i = i+1
    return dq


In [ ]:
#Suraj code main
#####################################################################################################################
## DEFINE DH PARAMETERS

# link1
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]
# plot scara
ax = plotrobotscara(DH) # to see plot need to do plt.show()
plt.show()
## OBTAIN JOINT TRAJECTORY USING END EFFECTOR TRAJECTORY
q0 = scara_inverse_kinematics(DH,0.4,0.01,0.1)
q1 = scara_inverse_kinematics(DH,0.4,0.06,0.1)
q1_poly = cubic(q0[0], q1[0], 0, 0, 0, 1)
q2_poly = cubic(q0[1], q1[1], 0, 0, 0, 1)
q3_poly = cubic(q0[2], q1[2], 0, 0, 0, 1)

# time from (t0,tfinal,no)
t=np.linspace(0,1,100)

q = np.transpose(np.zeros(6))
i = 0
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]  # sequence [theta,alpha,d,a]
# initial configuration
qi = scara_inverse_kinematics(DH,0.4,0.01,0.1)
#start state for integrator [q,dq] last three values indicate joint rate
q0 = [qi[0],qi[1],qi[2],0,0,0]
# motor and link parameter values
m1=1 ; m2=1 ; l1=1;l2=1; l3=1; I1=1/12; I2=1/12; I3=1/12; lc1=l1/2; lc2=l2/2; lc3=l3/2; g=9.81
Jm1 =0.4*10**-4; Jm2 =0.4*10**-4; Jm3=0.4*10**(-4); Bm1 = 4*10**(-5); Bm2 = 4*10**-5; Bm3=4*10**-5; Km1 =2*10**-2; Km2 =2*10**-2;Km3 =2*10**-2
Kb1 = 0.02;Kb2 = 0.02;Kb3=0.02;R1 = 0.3;R2 = 0.3;R3 = 0.3;r1 = 100;r2 = 100; r3=100; zeta = 1; wn = 50; K1 = 1; K2 = 1; K3 = 1

i = 0
### integrate the robot dynamics and control
q = integrate.odeint(modelscara,q0,t,args=(i, I1, I2, l1, l2,l3, lc1, lc2,lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2,r3, R1, R2,R3, Km1, Km2,Km3, Kb1, Kb2, Kb3,Bm1, Bm2, Bm3, zeta, wn,K1,K2,K3,q1_poly,q2_poly,q3_poly))

print("q",q)
xed = 0.4 * np.ones(100) # end-effector path only for plotting purpose
yed = np.linspace(0.01, 0.06, 100)
zed = 0.1 * np.ones(100)

DH_curr=[[q[:,0], alpha1, d1, a1*0.8], [q[:,1], alpha2, d2, a2], [theta3, alpha3, q[:,2], a3]]
xe,ye,ze=forward_kinematics2(DH_curr)


plt.figure(1)
tt = np.linspace(0,1,100)


# generate actual vs desired plots here

f=plt.figure(2)
for i in range(len(tt)):
  ax=f.add_subplot(111)
  ax.plot(tt,xed,color='r',label='Desired trajectory x')
  ax.plot(tt,xe,color='g',label='Actual trajectory x')

f1=plt.figure(3)
for i in range(len(tt)):
  ax=f1.add_subplot(111)
  ax.plot(tt,yed,color='r',label='Desired trajectory y')
  ax.plot(tt,ye,color='g',label='Actual trajectory y')
  
# Animate in 3D
for i in range(len(tt)):
    # substitute joint variables obtained from integration
    DH = [[q[i][0], alpha1, d1, a1], [q[i][1], alpha2, d2, a2], [theta3, alpha3, q[i][2], a3]]
    ax = plotrobotscara(DH)
    ax.plot3D(xed, yed, zed, '-m', label=r'$desired end-effector trajectory$')
    plt.draw()
    ax.text(0.5, 0.5, 0, 'step no ='+str(i), color='red')
    plt.pause(0.001)



##Q4 b) Independent joint control i.e. PD+Feedforward using motor dynamics with 20% error in length


In [ ]:
#Suraj Repo Dynamics + control
def modelscara(q, t, i, I1, I2, l1, l2, l3, lc1, lc2, lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2, r3, R1, R2, R3, Km1, Km2, Km3, Kb1, Kb2, Kb3,
               Bm1, Bm2, Bm3, zeta, wn, K1, K2, K3,q1_poly,q2_poly,q3_poly):

    dt = 0.01 # step size change accordingly
    t1 = q[0]
    t2 = q[1]
    t3 = q[2]
    #joint rates
    dt1 = q[3]
    dt2 = q[4]
    dt3 = q[5]

    # cubic desired trajectory
    qd1 = q1_poly[0]+q1_poly[1]*t+q1_poly[2]*t**2+q1_poly[3]*t**3
    qd2 = q2_poly[0] + q2_poly[1] * t + q2_poly[2] * t ** 2 + q2_poly[3] * t ** 3
    qd3 = q3_poly[0] + q3_poly[1] * t + q3_poly[2] * t ** 2 + q3_poly[3] * t ** 3
    # desired velocity
    dqd1 = q1_poly[1]  + 2*q1_poly[2] * t+ 3*q1_poly[3] * t **2
    dqd2 = q2_poly[1]  + 2*q2_poly[2] * t+ 3*q2_poly[3] * t **2
    dqd3 = q3_poly[1]  + 2*q3_poly[2] * t + 3*q3_poly[3] * t **2
    # desired acceleration
    ddqd1 =  2*q1_poly[2]+ 6*q1_poly[3] * t
    ddqd2 =  2*q2_poly[2]+ 6*q2_poly[3] * t
    ddqd3 =  2*q3_poly[2] + 6*q3_poly[3] * t

    # desired accl vector
    tdd = np.array([[ddqd1],[ddqd2],[ddqd2]])

    tdd = np.reshape(tdd,(3,1))
    # actual velocity vector
    td = np.array([[dt1],[dt2],[dt3]])

    tau = np.transpose(np.zeros(3))
    # Define Robot using PeterCorke toolbox just substitute non variable DH parameters Inertia and mass
    robot1 = rtb.DHRobot(
        [
            rtb.RevoluteDH(alpha=0, a=0.5, d=0.2, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.RevoluteDH(alpha=np.pi, a=0.5, d=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.PrismaticDH(theta=0, alpha=0, a=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
        ], name="scararob", gravity=[0, 0, 9.81])

    # Get D,C,phi
    D = robot1.inertia([t1,t2,t3])
    C = robot1.coriolis([t1,t2,t3],[dt1,dt2,dt3])
    phi = robot1.gravload([t1,t2,t3])
    phi = np.reshape(phi, (3, 1))

    #diagonal terms D to dkk
    dkk = np.array([[D[0][0], 0, 0],[0, D[1][1],0],[0, 0,D[2][2]]])
    # modelling disturbance, you can more disturbances here
    dk =   np.matmul(C, td) +np.matmul((D-dkk), tdd) +phi
    #motor parameters
    Jm = np.array([[Jm1,0,0],[0,Jm2,0],[0,0,Jm3]])
    Jeff1 = Jm1 + r1 * r1 * dkk[0][0]
    Jeff2 = Jm2 + r2 * r2 * dkk[1][1]
    Jeff3 = Jm3 + r3 * r3 * dkk[2][2]
    Beff1 = Bm1 + Kb1 * Km1 / R1
    Beff2 = Bm2 + Kb2 * Km2 / R2
    Beff3 = Bm3 + Kb3 * Km3 / R2
    K1 = Km1 / R1
    K2 = Km2 / R2
    K3 = Km3 / R3
    #gear ratio will assume same gear ratio so will only use scalar r1
    r = np.array([[r1, 0, 0],[0, r2,0],[0,0,r3]])
    Jeff = np.array([[Jeff1, 0,0],[0, Jeff2,0],[0,0,Jeff3]])
    Jeffinv = np.linalg.inv(Jeff)
    Beff = np.array([[Beff1, 0,0],[0, Beff2,0],[0,0,Beff3]])

    K = np.array([[K1,0,0],[0,K2,0],[0,0,K3]])/dt

    Kinv = np.linalg.inv(K)

    # compute errors
    e = np.array([[qd1-q[0]],[qd2-q[1]],[qd3-q[2]]])
    e = np.reshape(e,(3,1))
    ed = np.array([[0 - q[3]], [0 - q[4]], [0 - q[5]]])
    ed = np.reshape(e, (3, 1))
    
    #PID Gains
    f=np.matmul(np.matmul(Jeff,  Kinv),[ddqd1,ddqd2,ddqd3])+np.matmul(np.matmul(Beff,  Kinv),[dqd1,dqd2,dqd3])
    print(f)

    Kp = wn * wn * np.matmul(Jeff,  Kinv)
    
    Kd = np.matmul((2 * zeta * wn * Jeff - Beff ),Kinv) # change zeta here

    dt = 0.01
    # if i == 0:
    #     tprev = 0
    #     dt = t - tprev
    # else:
    #     dt = t - tprev
    #disp(size(dt));
    #sum = np.matmul(Ki, e) * dt

    V = np.matmul(Kp , e) + np.matmul(Kd ,ed) + f # voltage control input

    # dynamic equation in joint varaibles
    ddq = np.matmul(np.linalg.inv(Jeff),(r1*np.matmul(K ,V) - (r1**2)*(dk)- np.matmul(Beff , td)))
    # find statedot
    dq = [q[3],q[4],q[5],ddq[0][0],ddq[1][0],ddq[2][0]]

    #print('t= '+str(t))
    # tprev = t used for Integral
    # i = i+1
    return dq


In [ ]:
#Suraj code main
#####################################################################################################################
## DEFINE DH PARAMETERS

# link1
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]
# plot scara
ax = plotrobotscara(DH) # to see plot need to do plt.show()
plt.show()
## OBTAIN JOINT TRAJECTORY USING END EFFECTOR TRAJECTORY
q0 = scara_inverse_kinematics(DH,0.4,0.01,0.1)
q1 = scara_inverse_kinematics(DH,0.4,0.06,0.1)
q1_poly = cubic(q0[0], q1[0], 0, 0, 0, 1)
q2_poly = cubic(q0[1], q1[1], 0, 0, 0, 1)
q3_poly = cubic(q0[2], q1[2], 0, 0, 0, 1)

# time from (t0,tfinal,no)
t=np.linspace(0,1,100)

q = np.transpose(np.zeros(6))
i = 0
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]  # sequence [theta,alpha,d,a]
# initial configuration
qi = scara_inverse_kinematics(DH,0.4,0.01,0.1)
#start state for integrator [q,dq] last three values indicate joint rate
q0 = [qi[0],qi[1],qi[2],0,0,0]
# motor and link parameter values
m1=1 ; m2=1 ; l1=1;l2=1; l3=1; I1=1/12; I2=1/12; I3=1/12; lc1=l1/2; lc2=l2/2; lc3=l3/2; g=9.81
Jm1 =0.4*10**-4; Jm2 =0.4*10**-4; Jm3=0.4*10**(-4); Bm1 = 4*10**(-5); Bm2 = 4*10**-5; Bm3=4*10**-5; Km1 =2*10**-2; Km2 =2*10**-2;Km3 =2*10**-2
Kb1 = 0.02;Kb2 = 0.02;Kb3=0.02;R1 = 0.3;R2 = 0.3;R3 = 0.3;r1 = 100;r2 = 100; r3=100; zeta = 1; wn = 50; K1 = 1; K2 = 1; K3 = 1

i = 0
### integrate the robot dynamics and control
q = integrate.odeint(modelscara,q0,t,args=(i, I1, I2, l1, l2,l3, lc1, lc2,lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2,r3, R1, R2,R3, Km1, Km2,Km3, Kb1, Kb2, Kb3,Bm1, Bm2, Bm3, zeta, wn,K1,K2,K3,q1_poly,q2_poly,q3_poly))


xed = 0.4 * np.ones(100) # end-effector path only for plotting purpose
yed = np.linspace(0.01, 0.06, 100)
zed = 0.1 * np.ones(100)

DH_curr=[[q[:,0], alpha1, d1, 0.8*a1], [q[:,1], alpha2, d2, a2], [theta3, alpha3, q[:,2], a3]]
xe,ye,ze=forward_kinematics2(DH_curr)


plt.figure(1)
tt = np.linspace(0,1,100)


# generate actual vs desired plots here

f=plt.figure(2)
for i in range(len(tt)):
  ax=f.add_subplot(111)
  ax.plot(tt,xed,color='r',label='Desired trajectory x')
  ax.plot(tt,xe,color='g',label='Actual trajectory x')

f1=plt.figure(3)
for i in range(len(tt)):
  ax=f1.add_subplot(111)
  ax.plot(tt,yed,color='r',label='Desired trajectory y')
  ax.plot(tt,ye,color='g',label='Actual trajectory y')
  
# Animate in 3D
for i in range(len(tt)):
    # substitute joint variables obtained from integration
    DH = [[q[i][0], alpha1, d1, a1], [q[i][1], alpha2, d2, a2], [theta3, alpha3, q[i][2], a3]]
    ax = plotrobotscara(DH)
    ax.plot3D(xed, yed, zed, '-m', label=r'$desired end-effector trajectory$')
    plt.draw()
    ax.text(0.5, 0.5, 0, 'step no ='+str(i), color='red')
    plt.pause(0.001)



##Q4 c) Independent joint control i.e. PD+Feedforward using motor dynamics with computed torque with 20% error in length


In [ ]:
#Suraj Repo Dynamics + control
def modelscara(q, t, i, I1, I2, l1, l2, l3, lc1, lc2, lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2, r3, R1, R2, R3, Km1, Km2, Km3, Kb1, Kb2, Kb3,
               Bm1, Bm2, Bm3, zeta, wn, K1, K2, K3,q1_poly,q2_poly,q3_poly):

    dt = 0.01 # step size change accordingly
    t1 = q[0]
    t2 = q[1]
    t3 = q[2]
    #joint rates
    dt1 = q[3]
    dt2 = q[4]
    dt3 = q[5]

    # cubic desired trajectory
    qd1 = q1_poly[0]+q1_poly[1]*t+q1_poly[2]*t**2+q1_poly[3]*t**3
    qd2 = q2_poly[0] + q2_poly[1] * t + q2_poly[2] * t ** 2 + q2_poly[3] * t ** 3
    qd3 = q3_poly[0] + q3_poly[1] * t + q3_poly[2] * t ** 2 + q3_poly[3] * t ** 3
    # desired velocity
    dqd1 = q1_poly[1]  + 2*q1_poly[2] * t+ 3*q1_poly[3] * t **2
    dqd2 = q2_poly[1]  + 2*q2_poly[2] * t+ 3*q2_poly[3] * t **2
    dqd3 = q3_poly[1]  + 2*q3_poly[2] * t + 3*q3_poly[3] * t **2
    # desired acceleration
    ddqd1 =  2*q1_poly[2]+ 6*q1_poly[3] * t
    ddqd2 =  2*q2_poly[2]+ 6*q2_poly[3] * t
    ddqd3 =  2*q3_poly[2] + 6*q3_poly[3] * t

    # desired accl vector
    tdd = np.array([[ddqd1],[ddqd2],[ddqd2]])

    tdd = np.reshape(tdd,(3,1))
    # actual velocity vector
    td = np.array([[dt1],[dt2],[dt3]])

    tau = np.transpose(np.zeros(3))
    # Define Robot using PeterCorke toolbox just substitute non variable DH parameters Inertia and mass
    robot1 = rtb.DHRobot(
        [
            rtb.RevoluteDH(alpha=0, a=0.5, d=0.2, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.RevoluteDH(alpha=np.pi, a=0.5, d=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.PrismaticDH(theta=0, alpha=0, a=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
        ], name="scararob", gravity=[0, 0, 9.81])

    # Get D,C,phi
    D = robot1.inertia([t1,t2,t3])
    C = robot1.coriolis([t1,t2,t3],[dt1,dt2,dt3])
    phi = robot1.gravload([t1,t2,t3])
    phi = np.reshape(phi, (3, 1))

    #diagonal terms D to dkk
    dkk = np.array([[D[0][0], 0, 0],[0, D[1][1],0],[0, 0,D[2][2]]])
    # modelling disturbance, you can more disturbances here
    dk =   np.matmul(C, td) +np.matmul((D-dkk), tdd) +phi
    #motor parameters
    Jm = np.array([[Jm1,0,0],[0,Jm2,0],[0,0,Jm3]])
    Jeff1 = Jm1 + r1 * r1 * dkk[0][0]
    Jeff2 = Jm2 + r2 * r2 * dkk[1][1]
    Jeff3 = Jm3 + r3 * r3 * dkk[2][2]
    Beff1 = Bm1 + Kb1 * Km1 / R1
    Beff2 = Bm2 + Kb2 * Km2 / R2
    Beff3 = Bm3 + Kb3 * Km3 / R2
    K1 = Km1 / R1
    K2 = Km2 / R2
    K3 = Km3 / R3
    #gear ratio will assume same gear ratio so will only use scalar r1
    r = np.array([[r1, 0, 0],[0, r2,0],[0,0,r3]])
    Jeff = np.array([[Jeff1, 0,0],[0, Jeff2,0],[0,0,Jeff3]])
    Jeffinv = np.linalg.inv(Jeff)
    Beff = np.array([[Beff1, 0,0],[0, Beff2,0],[0,0,Beff3]])

    K = np.array([[K1,0,0],[0,K2,0],[0,0,K3]])/dt

    Kinv = np.linalg.inv(K)

    # compute errors
    e = np.array([[qd1-q[0]],[qd2-q[1]],[qd3-q[2]]])
    e = np.reshape(e,(3,1))
    ed = np.array([[0 - q[3]], [0 - q[4]], [0 - q[5]]])
    ed = np.reshape(e, (3, 1))
    
    #PID Gains
    f=np.matmul(np.matmul(Jeff,  Kinv),[ddqd1,ddqd2,ddqd3])+np.matmul(np.matmul(Beff,  Kinv),[dqd1,dqd2,dqd3])
    print(f)

    Kp = wn * wn * np.matmul(Jeff,  Kinv)
    
    Kd = np.matmul((2 * zeta * wn * Jeff - Beff ),Kinv) # change zeta here
    tau=dk

    dt = 0.01
    # if i == 0:
    #     tprev = 0
    #     dt = t - tprev
    # else:
    #     dt = t - tprev
    #disp(size(dt));
    #sum = np.matmul(Ki, e) * dt

    V = np.matmul(Kp , e) + np.matmul(Kd ,ed) + f +tau# voltage control input

    # dynamic equation in joint varaibles
    ddq = np.matmul(np.linalg.inv(Jeff),(r1*np.matmul(K ,V) - (r1**2)*(dk)- np.matmul(Beff , td)))
    # find statedot
    dq = [q[3],q[4],q[5],ddq[0][0],ddq[1][0],ddq[2][0]]

    #print('t= '+str(t))
    # tprev = t used for Integral
    # i = i+1
    return dq


In [ ]:
#Suraj code main
#####################################################################################################################
## DEFINE DH PARAMETERS

# link1
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]
# plot scara
ax = plotrobotscara(DH) # to see plot need to do plt.show()
plt.show()
## OBTAIN JOINT TRAJECTORY USING END EFFECTOR TRAJECTORY
q0 = scara_inverse_kinematics(DH,0.4,0.01,0.1)
q1 = scara_inverse_kinematics(DH,0.4,0.06,0.1)
q1_poly = cubic(q0[0], q1[0], 0, 0, 0, 1)
q2_poly = cubic(q0[1], q1[1], 0, 0, 0, 1)
q3_poly = cubic(q0[2], q1[2], 0, 0, 0, 1)

# time from (t0,tfinal,no)
t=np.linspace(0,1,100)

q = np.transpose(np.zeros(6))
i = 0
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]  # sequence [theta,alpha,d,a]
# initial configuration
qi = scara_inverse_kinematics(DH,0.4,0.01,0.1)
#start state for integrator [q,dq] last three values indicate joint rate
q0 = [qi[0],qi[1],qi[2],0,0,0]
# motor and link parameter values
m1=1 ; m2=1 ; l1=1;l2=1; l3=1; I1=1/12; I2=1/12; I3=1/12; lc1=l1/2; lc2=l2/2; lc3=l3/2; g=9.81
Jm1 =0.4*10**-4; Jm2 =0.4*10**-4; Jm3=0.4*10**(-4); Bm1 = 4*10**(-5); Bm2 = 4*10**-5; Bm3=4*10**-5; Km1 =2*10**-2; Km2 =2*10**-2;Km3 =2*10**-2
Kb1 = 0.02;Kb2 = 0.02;Kb3=0.02;R1 = 0.3;R2 = 0.3;R3 = 0.3;r1 = 100;r2 = 100; r3=100; zeta = 1; wn = 50; K1 = 1; K2 = 1; K3 = 1

i = 0
### integrate the robot dynamics and control
q = integrate.odeint(modelscara,q0,t,args=(i, I1, I2, l1, l2,l3, lc1, lc2,lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2,r3, R1, R2,R3, Km1, Km2,Km3, Kb1, Kb2, Kb3,Bm1, Bm2, Bm3, zeta, wn,K1,K2,K3,q1_poly,q2_poly,q3_poly))


xed = 0.4 * np.ones(100) # end-effector path only for plotting purpose
yed = np.linspace(0.01, 0.06, 100)
zed = 0.1 * np.ones(100)

DH_curr=[[q[:,0], alpha1, d1, 0.8*a1], [q[:,1], alpha2, d2, a2], [theta3, alpha3, q[:,2], a3]]
xe,ye,ze=forward_kinematics2(DH_curr)


plt.figure(1)
tt = np.linspace(0,1,100)


# generate actual vs desired plots here

f=plt.figure(2)
for i in range(len(tt)):
  ax=f.add_subplot(111)
  ax.plot(tt,xed,color='r',label='Desired trajectory x')
  ax.plot(tt,xe,color='g',label='Actual trajectory x')

f1=plt.figure(3)
for i in range(len(tt)):
  ax=f1.add_subplot(111)
  ax.plot(tt,yed,color='r',label='Desired trajectory y')
  ax.plot(tt,ye,color='g',label='Actual trajectory y')
  
# Animate in 3D
for i in range(len(tt)):
    # substitute joint variables obtained from integration
    DH = [[q[i][0], alpha1, d1, a1], [q[i][1], alpha2, d2, a2], [theta3, alpha3, q[i][2], a3]]
    ax = plotrobotscara(DH)
    ax.plot3D(xed, yed, zed, '-m', label=r'$desired end-effector trajectory$')
    plt.draw()
    ax.text(0.5, 0.5, 0, 'step no ='+str(i), color='red')
    plt.pause(0.001)



##Q5 a) PD control with $\zeta=1$ and random disturbance of SD 1Nm in dynamics

In [ ]:
#Suraj Repo Dynamics + control
def modelscara(q, t, i, I1, I2, l1, l2, l3, lc1, lc2, lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2, r3, R1, R2, R3, Km1, Km2, Km3, Kb1, Kb2, Kb3,
               Bm1, Bm2, Bm3, zeta, wn, K1, K2, K3,q1_poly,q2_poly,q3_poly):

    dt = 0.01 # step size change accordingly
    t1 = q[0]
    t2 = q[1]
    t3 = q[2]
    #joint rates
    dt1 = q[3]
    dt2 = q[4]
    dt3 = q[5]

    # cubic desired trajectory
    qd1 = q1_poly[0]+q1_poly[1]*t+q1_poly[2]*t**2+q1_poly[3]*t**3
    qd2 = q2_poly[0] + q2_poly[1] * t + q2_poly[2] * t ** 2 + q2_poly[3] * t ** 3
    qd3 = q3_poly[0] + q3_poly[1] * t + q3_poly[2] * t ** 2 + q3_poly[3] * t ** 3
    # desired velocity
    dqd1 = q1_poly[1]  + 2*q1_poly[2] * t+ 3*q1_poly[3] * t **2
    dqd2 = q2_poly[1]  + 2*q2_poly[2] * t+ 3*q2_poly[3] * t **2
    dqd3 = q3_poly[1]  + 2*q3_poly[2] * t + 3*q3_poly[3] * t **2
    # desired acceleration
    ddqd1 =  2*q1_poly[2]+ 6*q1_poly[3] * t
    ddqd2 =  2*q2_poly[2]+ 6*q2_poly[3] * t
    ddqd3 =  2*q3_poly[2] + 6*q3_poly[3] * t

    # desired accl vector
    tdd = np.array([[ddqd1],[ddqd2],[ddqd2]])

    tdd = np.reshape(tdd,(3,1))
    # actual velocity vector
    td = np.array([[dt1],[dt2],[dt3]])

    tau = np.transpose(np.zeros(3))
    # Define Robot using PeterCorke toolbox just substitute non variable DH parameters Inertia and mass
    robot1 = rtb.DHRobot(
        [
            rtb.RevoluteDH(alpha=0, a=0.5, d=0.2, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.RevoluteDH(alpha=np.pi, a=0.5, d=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.PrismaticDH(theta=0, alpha=0, a=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
        ], name="scararob", gravity=[0, 0, 9.81])

    # Get D,C,phi
    D = robot1.inertia([t1,t2,t3])
    C = robot1.coriolis([t1,t2,t3],[dt1,dt2,dt3])
    phi = robot1.gravload([t1,t2,t3])
    phi = np.reshape(phi, (3, 1))

    #diagonal terms D to dkk
    dkk = np.array([[D[0][0], 0, 0],[0, D[1][1],0],[0, 0,D[2][2]]])
    # modelling disturbance, you can more disturbances here
    dk =   np.matmul(C, td) +np.matmul((D-dkk), tdd) +phi
    #motor parameters
    Jm = np.array([[Jm1,0,0],[0,Jm2,0],[0,0,Jm3]])
    Jeff1 = Jm1 + r1 * r1 * dkk[0][0]
    Jeff2 = Jm2 + r2 * r2 * dkk[1][1]
    Jeff3 = Jm3 + r3 * r3 * dkk[2][2]
    Beff1 = Bm1 + Kb1 * Km1 / R1
    Beff2 = Bm2 + Kb2 * Km2 / R2
    Beff3 = Bm3 + Kb3 * Km3 / R2
    K1 = Km1 / R1
    K2 = Km2 / R2
    K3 = Km3 / R3
    #gear ratio will assume same gear ratio so will only use scalar r1
    r = np.array([[r1, 0, 0],[0, r2,0],[0,0,r3]])
    Jeff = np.array([[Jeff1, 0,0],[0, Jeff2,0],[0,0,Jeff3]])
    Jeffinv = np.linalg.inv(Jeff)
    Beff = np.array([[Beff1, 0,0],[0, Beff2,0],[0,0,Beff3]])

    K = np.array([[K1,0,0],[0,K2,0],[0,0,K3]])/dt

    Kinv = np.linalg.inv(K)

    # compute errors
    e = np.array([[qd1-q[0]],[qd2-q[1]],[qd3-q[2]]])
    e = np.reshape(e,(3,1))
    ed = np.array([[0 - q[3]], [0 - q[4]], [0 - q[5]]])
    ed = np.reshape(ed, (3, 1))
    
    #PID Gains
    Kp = wn * wn * np.matmul(Jeff,  Kinv)
    
    Kd = np.matmul((2 * zeta * wn * Jeff - Beff ),Kinv) # change zeta here

    dt = 0.01
    # if i == 0:
    #     tprev = 0
    #     dt = t - tprev
    # else:
    #     dt = t - tprev
    #disp(size(dt));
    #sum = np.matmul(Ki, e) * dt

    V = np.matmul(Kp , e) + np.matmul(Kd ,ed) # voltage control input

    # dynamic equation in joint varaibles
    ddq = np.matmul(np.linalg.inv(Jeff),(r1*np.matmul(K ,V) - (r1**2)*(dk)- np.matmul(Beff , td)))
    # find statedot
    dq = [q[3],q[4],q[5],ddq[0][0],ddq[1][0],ddq[2][0]]

    #print('t= '+str(t))
    # tprev = t used for Integral
    # i = i+1
    return dq


In [ ]:
#Suraj code main
#####################################################################################################################
## DEFINE DH PARAMETERS

# link1
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]
# plot scara
ax = plotrobotscara(DH) # to see plot need to do plt.show()
plt.show()
## OBTAIN JOINT TRAJECTORY USING END EFFECTOR TRAJECTORY
q0 = scara_inverse_kinematics(DH,0.4,0.01,0.1)
q1 = scara_inverse_kinematics(DH,0.4,0.06,0.1)
q1_poly = cubic(q0[0], q1[0], 0, 0, 0, 1)
q2_poly = cubic(q0[1], q1[1], 0, 0, 0, 1)
q3_poly = cubic(q0[2], q1[2], 0, 0, 0, 1)

# time from (t0,tfinal,no)
t=np.linspace(0,1,100)

q = np.transpose(np.zeros(6))
i = 0
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]  # sequence [theta,alpha,d,a]
# initial configuration
qi = scara_inverse_kinematics(DH,0.4,0.01,0.1)
#start state for integrator [q,dq] last three values indicate joint rate
q0 = [qi[0],qi[1],qi[2],0,0,0]
# motor and link parameter values
m1=1 ; m2=1 ; l1=1;l2=1; l3=1; I1=1/12; I2=1/12; I3=1/12; lc1=l1/2; lc2=l2/2; lc3=l3/2; g=9.81
Jm1 =0.4*10**-4; Jm2 =0.4*10**-4; Jm3=0.4*10**(-4); Bm1 = 4*10**(-5); Bm2 = 4*10**-5; Bm3=4*10**-5; Km1 =2*10**-2; Km2 =2*10**-2;Km3 =2*10**-2
Kb1 = 0.02;Kb2 = 0.02;Kb3=0.02;R1 = 0.3;R2 = 0.3;R3 = 0.3;r1 = 100;r2 = 100; r3=100; zeta = 1; wn = 50; K1 = 1; K2 = 1; K3 = 1

i = 0
### integrate the robot dynamics and control
q = integrate.odeint(modelscara,q0,t,args=(i, I1, I2, l1, l2,l3, lc1, lc2,lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2,r3, R1, R2,R3, Km1, Km2,Km3, Kb1, Kb2, Kb3,Bm1, Bm2, Bm3, zeta, wn,K1,K2,K3,q1_poly,q2_poly,q3_poly))

print("q",q)
xed = 0.4 * np.ones(100) # end-effector path only for plotting purpose
yed = np.linspace(0.01, 0.06, 100)
zed = 0.1 * np.ones(100)

DH_curr=[[q[:,0], alpha1, d1, a1], [q[:,1], alpha2, d2, a2], [theta3, alpha3, q[:,2], a3]]
xe,ye,ze=forward_kinematics2(DH_curr)


plt.figure(1)
tt = np.linspace(0,1,100)


# generate actual vs desired plots here

f=plt.figure(2)
for i in range(len(tt)):
  ax=f.add_subplot(111)
  ax.plot(tt,xed,color='r',label='Desired trajectory x')
  ax.plot(tt,xe,color='g',label='Actual trajectory x')

f1=plt.figure(3)
for i in range(len(tt)):
  ax=f1.add_subplot(111)
  ax.plot(tt,yed,color='r',label='Desired trajectory y')
  ax.plot(tt,ye,color='g',label='Actual trajectory y')
  
# Animate in 3D
for i in range(len(tt)):
    # substitute joint variables obtained from integration
    DH = [[q[i][0], alpha1, d1, a1], [q[i][1], alpha2, d2, a2], [theta3, alpha3, q[i][2], a3]]
    ax = plotrobotscara(DH)
    ax.plot3D(xed, yed, zed, '-m', label=r'$desired end-effector trajectory$')
    plt.draw()
    ax.text(0.5, 0.5, 0, 'step no ='+str(i), color='red')
    plt.pause(0.001)



##Q5 b) Independent joint control i.e. PD+Feedforward using motor dynamics and random disturbance of SD 1Nm in dynamics


In [ ]:
#Suraj Repo Dynamics + control
def modelscara(q, t, i, I1, I2, l1, l2, l3, lc1, lc2, lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2, r3, R1, R2, R3, Km1, Km2, Km3, Kb1, Kb2, Kb3,
               Bm1, Bm2, Bm3, zeta, wn, K1, K2, K3,q1_poly,q2_poly,q3_poly):

    dt = 0.01 # step size change accordingly
    t1 = q[0]
    t2 = q[1]
    t3 = q[2]
    #joint rates
    dt1 = q[3]
    dt2 = q[4]
    dt3 = q[5]

    # cubic desired trajectory
    qd1 = q1_poly[0]+q1_poly[1]*t+q1_poly[2]*t**2+q1_poly[3]*t**3
    qd2 = q2_poly[0] + q2_poly[1] * t + q2_poly[2] * t ** 2 + q2_poly[3] * t ** 3
    qd3 = q3_poly[0] + q3_poly[1] * t + q3_poly[2] * t ** 2 + q3_poly[3] * t ** 3
    # desired velocity
    dqd1 = q1_poly[1]  + 2*q1_poly[2] * t+ 3*q1_poly[3] * t **2
    dqd2 = q2_poly[1]  + 2*q2_poly[2] * t+ 3*q2_poly[3] * t **2
    dqd3 = q3_poly[1]  + 2*q3_poly[2] * t + 3*q3_poly[3] * t **2
    # desired acceleration
    ddqd1 =  2*q1_poly[2]+ 6*q1_poly[3] * t
    ddqd2 =  2*q2_poly[2]+ 6*q2_poly[3] * t
    ddqd3 =  2*q3_poly[2] + 6*q3_poly[3] * t

    # desired accl vector
    tdd = np.array([[ddqd1],[ddqd2],[ddqd2]])

    tdd = np.reshape(tdd,(3,1))
    # actual velocity vector
    td = np.array([[dt1],[dt2],[dt3]])

    tau = np.transpose(np.zeros(3))
    # Define Robot using PeterCorke toolbox just substitute non variable DH parameters Inertia and mass
    robot1 = rtb.DHRobot(
        [
            rtb.RevoluteDH(alpha=0, a=0.5, d=0.2, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.RevoluteDH(alpha=np.pi, a=0.5, d=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.PrismaticDH(theta=0, alpha=0, a=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
        ], name="scararob", gravity=[0, 0, 9.81])

    # Get D,C,phi
    D = robot1.inertia([t1,t2,t3])
    C = robot1.coriolis([t1,t2,t3],[dt1,dt2,dt3])
    phi = robot1.gravload([t1,t2,t3])
    phi = np.reshape(phi, (3, 1))

    #diagonal terms D to dkk
    dkk = np.array([[D[0][0], 0, 0],[0, D[1][1],0],[0, 0,D[2][2]]])
    # modelling disturbance, you can more disturbances here
    dk =   np.matmul(C, td) +np.matmul((D-dkk), tdd) +phi
    #motor parameters
    Jm = np.array([[Jm1,0,0],[0,Jm2,0],[0,0,Jm3]])
    Jeff1 = Jm1 + r1 * r1 * dkk[0][0]
    Jeff2 = Jm2 + r2 * r2 * dkk[1][1]
    Jeff3 = Jm3 + r3 * r3 * dkk[2][2]
    Beff1 = Bm1 + Kb1 * Km1 / R1
    Beff2 = Bm2 + Kb2 * Km2 / R2
    Beff3 = Bm3 + Kb3 * Km3 / R2
    K1 = Km1 / R1
    K2 = Km2 / R2
    K3 = Km3 / R3
    #gear ratio will assume same gear ratio so will only use scalar r1
    r = np.array([[r1, 0, 0],[0, r2,0],[0,0,r3]])
    Jeff = np.array([[Jeff1, 0,0],[0, Jeff2,0],[0,0,Jeff3]])
    Jeffinv = np.linalg.inv(Jeff)
    Beff = np.array([[Beff1, 0,0],[0, Beff2,0],[0,0,Beff3]])

    K = np.array([[K1,0,0],[0,K2,0],[0,0,K3]])/dt

    Kinv = np.linalg.inv(K)

    # compute errors
    e = np.array([[qd1-q[0]],[qd2-q[1]],[qd3-q[2]]])
    e = np.reshape(e,(3,1))
    ed = np.array([[0 - q[3]], [0 - q[4]], [0 - q[5]]])
    ed = np.reshape(e, (3, 1))
    
    #PID Gains
    f=np.matmul(np.matmul(Jeff,  Kinv),[ddqd1,ddqd2,ddqd3])+np.matmul(np.matmul(Beff,  Kinv),[dqd1,dqd2,dqd3])
    print(f)

    Kp = wn * wn * np.matmul(Jeff,  Kinv)
    
    Kd = np.matmul((2 * zeta * wn * Jeff - Beff ),Kinv) # change zeta here

    dt = 0.01
    # if i == 0:
    #     tprev = 0
    #     dt = t - tprev
    # else:
    #     dt = t - tprev
    #disp(size(dt));
    #sum = np.matmul(Ki, e) * dt

    V = np.matmul(Kp , e) + np.matmul(Kd ,ed) + f # voltage control input

    # dynamic equation in joint varaibles
    ddq = np.matmul(np.linalg.inv(Jeff),(r1*np.matmul(K ,V) - (r1**2)*(dk)- np.matmul(Beff , td)))
    # find statedot
    dq = [q[3],q[4],q[5],ddq[0][0],ddq[1][0],ddq[2][0]]

    #print('t= '+str(t))
    # tprev = t used for Integral
    # i = i+1
    return dq


In [ ]:
#Suraj code main
#####################################################################################################################
## DEFINE DH PARAMETERS

# link1
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]
# plot scara
ax = plotrobotscara(DH) # to see plot need to do plt.show()
plt.show()
## OBTAIN JOINT TRAJECTORY USING END EFFECTOR TRAJECTORY
q0 = scara_inverse_kinematics(DH,0.4,0.01,0.1)
q1 = scara_inverse_kinematics(DH,0.4,0.06,0.1)
q1_poly = cubic(q0[0], q1[0], 0, 0, 0, 1)
q2_poly = cubic(q0[1], q1[1], 0, 0, 0, 1)
q3_poly = cubic(q0[2], q1[2], 0, 0, 0, 1)

# time from (t0,tfinal,no)
t=np.linspace(0,1,100)

q = np.transpose(np.zeros(6))
i = 0
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]  # sequence [theta,alpha,d,a]
# initial configuration
qi = scara_inverse_kinematics(DH,0.4,0.01,0.1)
#start state for integrator [q,dq] last three values indicate joint rate
q0 = [qi[0],qi[1],qi[2],0,0,0]
# motor and link parameter values
m1=1 ; m2=1 ; l1=1;l2=1; l3=1; I1=1/12; I2=1/12; I3=1/12; lc1=l1/2; lc2=l2/2; lc3=l3/2; g=9.81
Jm1 =0.4*10**-4; Jm2 =0.4*10**-4; Jm3=0.4*10**(-4); Bm1 = 4*10**(-5); Bm2 = 4*10**-5; Bm3=4*10**-5; Km1 =2*10**-2; Km2 =2*10**-2;Km3 =2*10**-2
Kb1 = 0.02;Kb2 = 0.02;Kb3=0.02;R1 = 0.3;R2 = 0.3;R3 = 0.3;r1 = 100;r2 = 100; r3=100; zeta = 1; wn = 50; K1 = 1; K2 = 1; K3 = 1

i = 0
### integrate the robot dynamics and control
q = integrate.odeint(modelscara,q0,t,args=(i, I1, I2, l1, l2,l3, lc1, lc2,lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2,r3, R1, R2,R3, Km1, Km2,Km3, Kb1, Kb2, Kb3,Bm1, Bm2, Bm3, zeta, wn,K1,K2,K3,q1_poly,q2_poly,q3_poly))


xed = 0.4 * np.ones(100) # end-effector path only for plotting purpose
yed = np.linspace(0.01, 0.06, 100)
zed = 0.1 * np.ones(100)

DH_curr=[[q[:,0], alpha1, d1, a1], [q[:,1], alpha2, d2, a2], [theta3, alpha3, q[:,2], a3]]
xe,ye,ze=forward_kinematics2(DH_curr)


plt.figure(1)
tt = np.linspace(0,1,100)


# generate actual vs desired plots here

f=plt.figure(2)
for i in range(len(tt)):
  ax=f.add_subplot(111)
  ax.plot(tt,xed,color='r',label='Desired trajectory x')
  ax.plot(tt,xe,color='g',label='Actual trajectory x')

f1=plt.figure(3)
for i in range(len(tt)):
  ax=f1.add_subplot(111)
  ax.plot(tt,yed,color='r',label='Desired trajectory y')
  ax.plot(tt,ye,color='g',label='Actual trajectory y')
  
# Animate in 3D
for i in range(len(tt)):
    # substitute joint variables obtained from integration
    DH = [[q[i][0], alpha1, d1, a1], [q[i][1], alpha2, d2, a2], [theta3, alpha3, q[i][2], a3]]
    ax = plotrobotscara(DH)
    ax.plot3D(xed, yed, zed, '-m', label=r'$desired end-effector trajectory$')
    plt.draw()
    ax.text(0.5, 0.5, 0, 'step no ='+str(i), color='red')
    plt.pause(0.001)



##Q5 c) Independent joint control i.e. PD+Feedforward using motor dynamics with computed torque and random disturbance of SD 1Nm in dynamics


In [ ]:
#Suraj Repo Dynamics + control
def modelscara(q, t, i, I1, I2, l1, l2, l3, lc1, lc2, lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2, r3, R1, R2, R3, Km1, Km2, Km3, Kb1, Kb2, Kb3,
               Bm1, Bm2, Bm3, zeta, wn, K1, K2, K3,q1_poly,q2_poly,q3_poly):

    dt = 0.01 # step size change accordingly
    t1 = q[0]
    t2 = q[1]
    t3 = q[2]
    #joint rates
    dt1 = q[3]
    dt2 = q[4]
    dt3 = q[5]

    # cubic desired trajectory
    qd1 = q1_poly[0]+q1_poly[1]*t+q1_poly[2]*t**2+q1_poly[3]*t**3
    qd2 = q2_poly[0] + q2_poly[1] * t + q2_poly[2] * t ** 2 + q2_poly[3] * t ** 3
    qd3 = q3_poly[0] + q3_poly[1] * t + q3_poly[2] * t ** 2 + q3_poly[3] * t ** 3
    # desired velocity
    dqd1 = q1_poly[1]  + 2*q1_poly[2] * t+ 3*q1_poly[3] * t **2
    dqd2 = q2_poly[1]  + 2*q2_poly[2] * t+ 3*q2_poly[3] * t **2
    dqd3 = q3_poly[1]  + 2*q3_poly[2] * t + 3*q3_poly[3] * t **2
    # desired acceleration
    ddqd1 =  2*q1_poly[2]+ 6*q1_poly[3] * t
    ddqd2 =  2*q2_poly[2]+ 6*q2_poly[3] * t
    ddqd3 =  2*q3_poly[2] + 6*q3_poly[3] * t

    # desired accl vector
    tdd = np.array([[ddqd1],[ddqd2],[ddqd2]])

    tdd = np.reshape(tdd,(3,1))
    # actual velocity vector
    td = np.array([[dt1],[dt2],[dt3]])

    tau = np.transpose(np.zeros(3))
    # Define Robot using PeterCorke toolbox just substitute non variable DH parameters Inertia and mass
    robot1 = rtb.DHRobot(
        [
            rtb.RevoluteDH(alpha=0, a=0.5, d=0.2, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.RevoluteDH(alpha=np.pi, a=0.5, d=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.PrismaticDH(theta=0, alpha=0, a=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
        ], name="scararob", gravity=[0, 0, 9.81])

    # Get D,C,phi
    D = robot1.inertia([t1,t2,t3])
    C = robot1.coriolis([t1,t2,t3],[dt1,dt2,dt3])
    phi = robot1.gravload([t1,t2,t3])
    phi = np.reshape(phi, (3, 1))

    #diagonal terms D to dkk
    dkk = np.array([[D[0][0], 0, 0],[0, D[1][1],0],[0, 0,D[2][2]]])
    # modelling disturbance, you can more disturbances here
    dk =   np.matmul(C, td) +np.matmul((D-dkk), tdd) +phi
    #motor parameters
    Jm = np.array([[Jm1,0,0],[0,Jm2,0],[0,0,Jm3]])
    Jeff1 = Jm1 + r1 * r1 * dkk[0][0]
    Jeff2 = Jm2 + r2 * r2 * dkk[1][1]
    Jeff3 = Jm3 + r3 * r3 * dkk[2][2]
    Beff1 = Bm1 + Kb1 * Km1 / R1
    Beff2 = Bm2 + Kb2 * Km2 / R2
    Beff3 = Bm3 + Kb3 * Km3 / R2
    K1 = Km1 / R1
    K2 = Km2 / R2
    K3 = Km3 / R3
    #gear ratio will assume same gear ratio so will only use scalar r1
    r = np.array([[r1, 0, 0],[0, r2,0],[0,0,r3]])
    Jeff = np.array([[Jeff1, 0,0],[0, Jeff2,0],[0,0,Jeff3]])
    Jeffinv = np.linalg.inv(Jeff)
    Beff = np.array([[Beff1, 0,0],[0, Beff2,0],[0,0,Beff3]])

    K = np.array([[K1,0,0],[0,K2,0],[0,0,K3]])/dt

    Kinv = np.linalg.inv(K)

    # compute errors
    e = np.array([[qd1-q[0]],[qd2-q[1]],[qd3-q[2]]])
    e = np.reshape(e,(3,1))
    ed = np.array([[0 - q[3]], [0 - q[4]], [0 - q[5]]])
    ed = np.reshape(e, (3, 1))
    
    #PID Gains
    f=np.matmul(np.matmul(Jeff,  Kinv),[ddqd1,ddqd2,ddqd3])+np.matmul(np.matmul(Beff,  Kinv),[dqd1,dqd2,dqd3])
    print(f)

    Kp = wn * wn * np.matmul(Jeff,  Kinv)
    
    Kd = np.matmul((2 * zeta * wn * Jeff - Beff ),Kinv) # change zeta here
    tau=dk

    dt = 0.01
    # if i == 0:
    #     tprev = 0
    #     dt = t - tprev
    # else:
    #     dt = t - tprev
    #disp(size(dt));
    #sum = np.matmul(Ki, e) * dt

    V = np.matmul(Kp , e) + np.matmul(Kd ,ed) + f +tau# voltage control input

    # dynamic equation in joint varaibles
    ddq = np.matmul(np.linalg.inv(Jeff),(r1*np.matmul(K ,V) - (r1**2)*(dk)- np.matmul(Beff , td)))
    # find statedot
    dq = [q[3],q[4],q[5],ddq[0][0],ddq[1][0],ddq[2][0]]

    #print('t= '+str(t))
    # tprev = t used for Integral
    # i = i+1
    return dq


In [ ]:
#Suraj code main
#####################################################################################################################
## DEFINE DH PARAMETERS

# link1
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]
# plot scara
ax = plotrobotscara(DH) # to see plot need to do plt.show()
plt.show()
## OBTAIN JOINT TRAJECTORY USING END EFFECTOR TRAJECTORY
q0 = scara_inverse_kinematics(DH,0.4,0.01,0.1)
q1 = scara_inverse_kinematics(DH,0.4,0.06,0.1)
q1_poly = cubic(q0[0], q1[0], 0, 0, 0, 1)
q2_poly = cubic(q0[1], q1[1], 0, 0, 0, 1)
q3_poly = cubic(q0[2], q1[2], 0, 0, 0, 1)

# time from (t0,tfinal,no)
t=np.linspace(0,1,100)

q = np.transpose(np.zeros(6))
i = 0
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]  # sequence [theta,alpha,d,a]
# initial configuration
qi = scara_inverse_kinematics(DH,0.4,0.01,0.1)
#start state for integrator [q,dq] last three values indicate joint rate
q0 = [qi[0],qi[1],qi[2],0,0,0]
# motor and link parameter values
m1=1 ; m2=1 ; l1=1;l2=1; l3=1; I1=1/12; I2=1/12; I3=1/12; lc1=l1/2; lc2=l2/2; lc3=l3/2; g=9.81
Jm1 =0.4*10**-4; Jm2 =0.4*10**-4; Jm3=0.4*10**(-4); Bm1 = 4*10**(-5); Bm2 = 4*10**-5; Bm3=4*10**-5; Km1 =2*10**-2; Km2 =2*10**-2;Km3 =2*10**-2
Kb1 = 0.02;Kb2 = 0.02;Kb3=0.02;R1 = 0.3;R2 = 0.3;R3 = 0.3;r1 = 100;r2 = 100; r3=100; zeta = 1; wn = 50; K1 = 1; K2 = 1; K3 = 1

i = 0
### integrate the robot dynamics and control
q = integrate.odeint(modelscara,q0,t,args=(i, I1, I2, l1, l2,l3, lc1, lc2,lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2,r3, R1, R2,R3, Km1, Km2,Km3, Kb1, Kb2, Kb3,Bm1, Bm2, Bm3, zeta, wn,K1,K2,K3,q1_poly,q2_poly,q3_poly))


xed = 0.4 * np.ones(100) # end-effector path only for plotting purpose
yed = np.linspace(0.01, 0.06, 100)
zed = 0.1 * np.ones(100)

DH_curr=[[q[:,0], alpha1, d1, a1], [q[:,1], alpha2, d2, a2], [theta3, alpha3, q[:,2], a3]]
xe,ye,ze=forward_kinematics2(DH_curr)


plt.figure(1)
tt = np.linspace(0,1,100)


# generate actual vs desired plots here

f=plt.figure(2)
for i in range(len(tt)):
  ax=f.add_subplot(111)
  ax.plot(tt,xed,color='r',label='Desired trajectory x')
  ax.plot(tt,xe,color='g',label='Actual trajectory x')

f1=plt.figure(3)
for i in range(len(tt)):
  ax=f1.add_subplot(111)
  ax.plot(tt,yed,color='r',label='Desired trajectory y')
  ax.plot(tt,ye,color='g',label='Actual trajectory y')
  
# Animate in 3D
for i in range(len(tt)):
    # substitute joint variables obtained from integration
    DH = [[q[i][0], alpha1, d1, a1], [q[i][1], alpha2, d2, a2], [theta3, alpha3, q[i][2], a3]]
    ax = plotrobotscara(DH)
    ax.plot3D(xed, yed, zed, '-m', label=r'$desired end-effector trajectory$')
    plt.draw()
    ax.text(0.5, 0.5, 0, 'step no ='+str(i), color='red')
    plt.pause(0.001)



##Q6 a) PD control with $\zeta=1$ and impulsive disturbance in dynamics

In [ ]:
#Suraj Repo Dynamics + control
def modelscara(q, t, i, I1, I2, l1, l2, l3, lc1, lc2, lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2, r3, R1, R2, R3, Km1, Km2, Km3, Kb1, Kb2, Kb3,
               Bm1, Bm2, Bm3, zeta, wn, K1, K2, K3,q1_poly,q2_poly,q3_poly):

    dt = 0.01 # step size change accordingly
    t1 = q[0]
    t2 = q[1]
    t3 = q[2]
    #joint rates
    dt1 = q[3]
    dt2 = q[4]
    dt3 = q[5]

    # cubic desired trajectory
    qd1 = q1_poly[0]+q1_poly[1]*t+q1_poly[2]*t**2+q1_poly[3]*t**3
    qd2 = q2_poly[0] + q2_poly[1] * t + q2_poly[2] * t ** 2 + q2_poly[3] * t ** 3
    qd3 = q3_poly[0] + q3_poly[1] * t + q3_poly[2] * t ** 2 + q3_poly[3] * t ** 3
    # desired velocity
    dqd1 = q1_poly[1]  + 2*q1_poly[2] * t+ 3*q1_poly[3] * t **2
    dqd2 = q2_poly[1]  + 2*q2_poly[2] * t+ 3*q2_poly[3] * t **2
    dqd3 = q3_poly[1]  + 2*q3_poly[2] * t + 3*q3_poly[3] * t **2
    # desired acceleration
    ddqd1 =  2*q1_poly[2]+ 6*q1_poly[3] * t
    ddqd2 =  2*q2_poly[2]+ 6*q2_poly[3] * t
    ddqd3 =  2*q3_poly[2] + 6*q3_poly[3] * t

    # desired accl vector
    tdd = np.array([[ddqd1],[ddqd2],[ddqd2]])

    tdd = np.reshape(tdd,(3,1))
    # actual velocity vector
    td = np.array([[dt1],[dt2],[dt3]])

    tau = np.transpose(np.zeros(3))
    # Define Robot using PeterCorke toolbox just substitute non variable DH parameters Inertia and mass
    robot1 = rtb.DHRobot(
        [
            rtb.RevoluteDH(alpha=0, a=0.5, d=0.2, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.RevoluteDH(alpha=np.pi, a=0.5, d=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.PrismaticDH(theta=0, alpha=0, a=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
        ], name="scararob", gravity=[0, 0, 9.81])

    # Get D,C,phi
    D = robot1.inertia([t1,t2,t3])
    C = robot1.coriolis([t1,t2,t3],[dt1,dt2,dt3])
    phi = robot1.gravload([t1,t2,t3])
    phi = np.reshape(phi, (3, 1))

    #diagonal terms D to dkk
    dkk = np.array([[D[0][0], 0, 0],[0, D[1][1],0],[0, 0,D[2][2]]])
    # modelling disturbance, you can more disturbances here
    dk =   np.matmul(C, td) +np.matmul((D-dkk), tdd) +phi
    #motor parameters
    Jm = np.array([[Jm1,0,0],[0,Jm2,0],[0,0,Jm3]])
    Jeff1 = Jm1 + r1 * r1 * dkk[0][0]
    Jeff2 = Jm2 + r2 * r2 * dkk[1][1]
    Jeff3 = Jm3 + r3 * r3 * dkk[2][2]
    Beff1 = Bm1 + Kb1 * Km1 / R1
    Beff2 = Bm2 + Kb2 * Km2 / R2
    Beff3 = Bm3 + Kb3 * Km3 / R2
    K1 = Km1 / R1
    K2 = Km2 / R2
    K3 = Km3 / R3
    #gear ratio will assume same gear ratio so will only use scalar r1
    r = np.array([[r1, 0, 0],[0, r2,0],[0,0,r3]])
    Jeff = np.array([[Jeff1, 0,0],[0, Jeff2,0],[0,0,Jeff3]])
    Jeffinv = np.linalg.inv(Jeff)
    Beff = np.array([[Beff1, 0,0],[0, Beff2,0],[0,0,Beff3]])

    K = np.array([[K1,0,0],[0,K2,0],[0,0,K3]])/dt

    Kinv = np.linalg.inv(K)

    # compute errors
    e = np.array([[qd1-q[0]],[qd2-q[1]],[qd3-q[2]]])
    e = np.reshape(e,(3,1))
    ed = np.array([[0 - q[3]], [0 - q[4]], [0 - q[5]]])
    ed = np.reshape(ed, (3, 1))
    
    #PID Gains
    Kp = wn * wn * np.matmul(Jeff,  Kinv)
    
    Kd = np.matmul((2 * zeta * wn * Jeff - Beff ),Kinv) # change zeta here

    dt = 0.01
    # if i == 0:
    #     tprev = 0
    #     dt = t - tprev
    # else:
    #     dt = t - tprev
    #disp(size(dt));
    #sum = np.matmul(Ki, e) * dt

    V = np.matmul(Kp , e) + np.matmul(Kd ,ed) # voltage control input

    # dynamic equation in joint varaibles
    ddq = np.matmul(np.linalg.inv(Jeff),(r1*np.matmul(K ,V) - (r1**2)*(dk)- np.matmul(Beff , td)))
    # find statedot
    dq = [q[3],q[4],q[5],ddq[0][0],ddq[1][0],ddq[2][0]]

    #Impulse

    if t==5:
      dq=[100,100,100,10000,10000,10000]

    #print('t= '+str(t))
    # tprev = t used for Integral
    # i = i+1
    return dq


In [ ]:
#Suraj code main
#####################################################################################################################
## DEFINE DH PARAMETERS

# link1
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]
# plot scara
ax = plotrobotscara(DH) # to see plot need to do plt.show()
plt.show()
## OBTAIN JOINT TRAJECTORY USING END EFFECTOR TRAJECTORY
q0 = scara_inverse_kinematics(DH,0.4,0.01,0.1)
q1 = scara_inverse_kinematics(DH,0.4,0.06,0.1)
q1_poly = cubic(q0[0], q1[0], 0, 0, 0, 1)
q2_poly = cubic(q0[1], q1[1], 0, 0, 0, 1)
q3_poly = cubic(q0[2], q1[2], 0, 0, 0, 1)

# time from (t0,tfinal,no)
t=np.linspace(0,1,100)

q = np.transpose(np.zeros(6))
i = 0
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]  # sequence [theta,alpha,d,a]
# initial configuration
qi = scara_inverse_kinematics(DH,0.4,0.01,0.1)
#start state for integrator [q,dq] last three values indicate joint rate
q0 = [qi[0],qi[1],qi[2],0,0,0]
# motor and link parameter values
m1=1 ; m2=1 ; l1=1;l2=1; l3=1; I1=1/12; I2=1/12; I3=1/12; lc1=l1/2; lc2=l2/2; lc3=l3/2; g=9.81
Jm1 =0.4*10**-4; Jm2 =0.4*10**-4; Jm3=0.4*10**(-4); Bm1 = 4*10**(-5); Bm2 = 4*10**-5; Bm3=4*10**-5; Km1 =2*10**-2; Km2 =2*10**-2;Km3 =2*10**-2
Kb1 = 0.02;Kb2 = 0.02;Kb3=0.02;R1 = 0.3;R2 = 0.3;R3 = 0.3;r1 = 100;r2 = 100; r3=100; zeta = 1; wn = 50; K1 = 1; K2 = 1; K3 = 1

i = 0
### integrate the robot dynamics and control
q = integrate.odeint(modelscara,q0,t,args=(i, I1, I2, l1, l2,l3, lc1, lc2,lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2,r3, R1, R2,R3, Km1, Km2,Km3, Kb1, Kb2, Kb3,Bm1, Bm2, Bm3, zeta, wn,K1,K2,K3,q1_poly,q2_poly,q3_poly))

print("q",q)
xed = 0.4 * np.ones(100) # end-effector path only for plotting purpose
yed = np.linspace(0.01, 0.06, 100)
zed = 0.1 * np.ones(100)

DH_curr=[[q[:,0], alpha1, d1, a1], [q[:,1], alpha2, d2, a2], [theta3, alpha3, q[:,2], a3]]
xe,ye,ze=forward_kinematics2(DH_curr)


plt.figure(1)
tt = np.linspace(0,1,100)


# generate actual vs desired plots here

f=plt.figure(2)
for i in range(len(tt)):
  ax=f.add_subplot(111)
  ax.plot(tt,xed,color='r',label='Desired trajectory x')
  ax.plot(tt,xe,color='g',label='Actual trajectory x')

f1=plt.figure(3)
for i in range(len(tt)):
  ax=f1.add_subplot(111)
  ax.plot(tt,yed,color='r',label='Desired trajectory y')
  ax.plot(tt,ye,color='g',label='Actual trajectory y')
  
# Animate in 3D
for i in range(len(tt)):
    # substitute joint variables obtained from integration
    DH = [[q[i][0], alpha1, d1, a1], [q[i][1], alpha2, d2, a2], [theta3, alpha3, q[i][2], a3]]
    ax = plotrobotscara(DH)
    ax.plot3D(xed, yed, zed, '-m', label=r'$desired end-effector trajectory$')
    plt.draw()
    ax.text(0.5, 0.5, 0, 'step no ='+str(i), color='red')
    plt.pause(0.001)



##Q6 b) Independent joint control i.e. PD+Feedforward using motor dynamics  and impulsive disturbance in dynamics


In [ ]:
#Suraj Repo Dynamics + control
def modelscara(q, t, i, I1, I2, l1, l2, l3, lc1, lc2, lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2, r3, R1, R2, R3, Km1, Km2, Km3, Kb1, Kb2, Kb3,
               Bm1, Bm2, Bm3, zeta, wn, K1, K2, K3,q1_poly,q2_poly,q3_poly):

    dt = 0.01 # step size change accordingly
    t1 = q[0]
    t2 = q[1]
    t3 = q[2]
    #joint rates
    dt1 = q[3]
    dt2 = q[4]
    dt3 = q[5]

    # cubic desired trajectory
    qd1 = q1_poly[0]+q1_poly[1]*t+q1_poly[2]*t**2+q1_poly[3]*t**3
    qd2 = q2_poly[0] + q2_poly[1] * t + q2_poly[2] * t ** 2 + q2_poly[3] * t ** 3
    qd3 = q3_poly[0] + q3_poly[1] * t + q3_poly[2] * t ** 2 + q3_poly[3] * t ** 3
    # desired velocity
    dqd1 = q1_poly[1]  + 2*q1_poly[2] * t+ 3*q1_poly[3] * t **2
    dqd2 = q2_poly[1]  + 2*q2_poly[2] * t+ 3*q2_poly[3] * t **2
    dqd3 = q3_poly[1]  + 2*q3_poly[2] * t + 3*q3_poly[3] * t **2
    # desired acceleration
    ddqd1 =  2*q1_poly[2]+ 6*q1_poly[3] * t
    ddqd2 =  2*q2_poly[2]+ 6*q2_poly[3] * t
    ddqd3 =  2*q3_poly[2] + 6*q3_poly[3] * t

    # desired accl vector
    tdd = np.array([[ddqd1],[ddqd2],[ddqd2]])

    tdd = np.reshape(tdd,(3,1))
    # actual velocity vector
    td = np.array([[dt1],[dt2],[dt3]])

    tau = np.transpose(np.zeros(3))
    # Define Robot using PeterCorke toolbox just substitute non variable DH parameters Inertia and mass
    robot1 = rtb.DHRobot(
        [
            rtb.RevoluteDH(alpha=0, a=0.5, d=0.2, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.RevoluteDH(alpha=np.pi, a=0.5, d=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.PrismaticDH(theta=0, alpha=0, a=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
        ], name="scararob", gravity=[0, 0, 9.81])

    # Get D,C,phi
    D = robot1.inertia([t1,t2,t3])
    C = robot1.coriolis([t1,t2,t3],[dt1,dt2,dt3])
    phi = robot1.gravload([t1,t2,t3])
    phi = np.reshape(phi, (3, 1))

    #diagonal terms D to dkk
    dkk = np.array([[D[0][0], 0, 0],[0, D[1][1],0],[0, 0,D[2][2]]])
    # modelling disturbance, you can more disturbances here
    dk =   np.matmul(C, td) +np.matmul((D-dkk), tdd) +phi
    #motor parameters
    Jm = np.array([[Jm1,0,0],[0,Jm2,0],[0,0,Jm3]])
    Jeff1 = Jm1 + r1 * r1 * dkk[0][0]
    Jeff2 = Jm2 + r2 * r2 * dkk[1][1]
    Jeff3 = Jm3 + r3 * r3 * dkk[2][2]
    Beff1 = Bm1 + Kb1 * Km1 / R1
    Beff2 = Bm2 + Kb2 * Km2 / R2
    Beff3 = Bm3 + Kb3 * Km3 / R2
    K1 = Km1 / R1
    K2 = Km2 / R2
    K3 = Km3 / R3
    #gear ratio will assume same gear ratio so will only use scalar r1
    r = np.array([[r1, 0, 0],[0, r2,0],[0,0,r3]])
    Jeff = np.array([[Jeff1, 0,0],[0, Jeff2,0],[0,0,Jeff3]])
    Jeffinv = np.linalg.inv(Jeff)
    Beff = np.array([[Beff1, 0,0],[0, Beff2,0],[0,0,Beff3]])

    K = np.array([[K1,0,0],[0,K2,0],[0,0,K3]])/dt

    Kinv = np.linalg.inv(K)

    # compute errors
    e = np.array([[qd1-q[0]],[qd2-q[1]],[qd3-q[2]]])
    e = np.reshape(e,(3,1))
    ed = np.array([[0 - q[3]], [0 - q[4]], [0 - q[5]]])
    ed = np.reshape(e, (3, 1))
    
    #PID Gains
    f=np.matmul(np.matmul(Jeff,  Kinv),[ddqd1,ddqd2,ddqd3])+np.matmul(np.matmul(Beff,  Kinv),[dqd1,dqd2,dqd3])
    print(f)

    Kp = wn * wn * np.matmul(Jeff,  Kinv)
    
    Kd = np.matmul((2 * zeta * wn * Jeff - Beff ),Kinv) # change zeta here

    dt = 0.01
    # if i == 0:
    #     tprev = 0
    #     dt = t - tprev
    # else:
    #     dt = t - tprev
    #disp(size(dt));
    #sum = np.matmul(Ki, e) * dt

    V = np.matmul(Kp , e) + np.matmul(Kd ,ed) + f # voltage control input

    # dynamic equation in joint varaibles
    ddq = np.matmul(np.linalg.inv(Jeff),(r1*np.matmul(K ,V) - (r1**2)*(dk)- np.matmul(Beff , td)))
    # find statedot
    dq = [q[3],q[4],q[5],ddq[0][0],ddq[1][0],ddq[2][0]]
    
    if t==5:
      dq=[100,100,100,10000,10000,10000]

    #print('t= '+str(t))
    # tprev = t used for Integral
    # i = i+1
    return dq


In [ ]:
#Suraj code main
#####################################################################################################################
## DEFINE DH PARAMETERS

# link1
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]
# plot scara
ax = plotrobotscara(DH) # to see plot need to do plt.show()
plt.show()
## OBTAIN JOINT TRAJECTORY USING END EFFECTOR TRAJECTORY
q0 = scara_inverse_kinematics(DH,0.4,0.01,0.1)
q1 = scara_inverse_kinematics(DH,0.4,0.06,0.1)
q1_poly = cubic(q0[0], q1[0], 0, 0, 0, 1)
q2_poly = cubic(q0[1], q1[1], 0, 0, 0, 1)
q3_poly = cubic(q0[2], q1[2], 0, 0, 0, 1)

# time from (t0,tfinal,no)
t=np.linspace(0,1,100)

q = np.transpose(np.zeros(6))
i = 0
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]  # sequence [theta,alpha,d,a]
# initial configuration
qi = scara_inverse_kinematics(DH,0.4,0.01,0.1)
#start state for integrator [q,dq] last three values indicate joint rate
q0 = [qi[0],qi[1],qi[2],0,0,0]
# motor and link parameter values
m1=1 ; m2=1 ; l1=1;l2=1; l3=1; I1=1/12; I2=1/12; I3=1/12; lc1=l1/2; lc2=l2/2; lc3=l3/2; g=9.81
Jm1 =0.4*10**-4; Jm2 =0.4*10**-4; Jm3=0.4*10**(-4); Bm1 = 4*10**(-5); Bm2 = 4*10**-5; Bm3=4*10**-5; Km1 =2*10**-2; Km2 =2*10**-2;Km3 =2*10**-2
Kb1 = 0.02;Kb2 = 0.02;Kb3=0.02;R1 = 0.3;R2 = 0.3;R3 = 0.3;r1 = 100;r2 = 100; r3=100; zeta = 1; wn = 50; K1 = 1; K2 = 1; K3 = 1

i = 0
### integrate the robot dynamics and control
q = integrate.odeint(modelscara,q0,t,args=(i, I1, I2, l1, l2,l3, lc1, lc2,lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2,r3, R1, R2,R3, Km1, Km2,Km3, Kb1, Kb2, Kb3,Bm1, Bm2, Bm3, zeta, wn,K1,K2,K3,q1_poly,q2_poly,q3_poly))


xed = 0.4 * np.ones(100) # end-effector path only for plotting purpose
yed = np.linspace(0.01, 0.06, 100)
zed = 0.1 * np.ones(100)

DH_curr=[[q[:,0], alpha1, d1, a1], [q[:,1], alpha2, d2, a2], [theta3, alpha3, q[:,2], a3]]
xe,ye,ze=forward_kinematics2(DH_curr)


plt.figure(1)
tt = np.linspace(0,1,100)


# generate actual vs desired plots here

f=plt.figure(2)
for i in range(len(tt)):
  ax=f.add_subplot(111)
  ax.plot(tt,xed,color='r',label='Desired trajectory x')
  ax.plot(tt,xe,color='g',label='Actual trajectory x')

f1=plt.figure(3)
for i in range(len(tt)):
  ax=f1.add_subplot(111)
  ax.plot(tt,yed,color='r',label='Desired trajectory y')
  ax.plot(tt,ye,color='g',label='Actual trajectory y')
  
# Animate in 3D
for i in range(len(tt)):
    # substitute joint variables obtained from integration
    DH = [[q[i][0], alpha1, d1, a1], [q[i][1], alpha2, d2, a2], [theta3, alpha3, q[i][2], a3]]
    ax = plotrobotscara(DH)
    ax.plot3D(xed, yed, zed, '-m', label=r'$desired end-effector trajectory$')
    plt.draw()
    ax.text(0.5, 0.5, 0, 'step no ='+str(i), color='red')
    plt.pause(0.001)



##Q6 c) Independent joint control i.e. PD+Feedforward using motor dynamics with computed torque  and impulsive disturbance in dynamics


In [ ]:
#Suraj Repo Dynamics + control
def modelscara(q, t, i, I1, I2, l1, l2, l3, lc1, lc2, lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2, r3, R1, R2, R3, Km1, Km2, Km3, Kb1, Kb2, Kb3,
               Bm1, Bm2, Bm3, zeta, wn, K1, K2, K3,q1_poly,q2_poly,q3_poly):

    dt = 0.01 # step size change accordingly
    t1 = q[0]
    t2 = q[1]
    t3 = q[2]
    #joint rates
    dt1 = q[3]
    dt2 = q[4]
    dt3 = q[5]

    # cubic desired trajectory
    qd1 = q1_poly[0]+q1_poly[1]*t+q1_poly[2]*t**2+q1_poly[3]*t**3
    qd2 = q2_poly[0] + q2_poly[1] * t + q2_poly[2] * t ** 2 + q2_poly[3] * t ** 3
    qd3 = q3_poly[0] + q3_poly[1] * t + q3_poly[2] * t ** 2 + q3_poly[3] * t ** 3
    # desired velocity
    dqd1 = q1_poly[1]  + 2*q1_poly[2] * t+ 3*q1_poly[3] * t **2
    dqd2 = q2_poly[1]  + 2*q2_poly[2] * t+ 3*q2_poly[3] * t **2
    dqd3 = q3_poly[1]  + 2*q3_poly[2] * t + 3*q3_poly[3] * t **2
    # desired acceleration
    ddqd1 =  2*q1_poly[2]+ 6*q1_poly[3] * t
    ddqd2 =  2*q2_poly[2]+ 6*q2_poly[3] * t
    ddqd3 =  2*q3_poly[2] + 6*q3_poly[3] * t

    # desired accl vector
    tdd = np.array([[ddqd1],[ddqd2],[ddqd2]])

    tdd = np.reshape(tdd,(3,1))
    # actual velocity vector
    td = np.array([[dt1],[dt2],[dt3]])

    tau = np.transpose(np.zeros(3))
    # Define Robot using PeterCorke toolbox just substitute non variable DH parameters Inertia and mass
    robot1 = rtb.DHRobot(
        [
            rtb.RevoluteDH(alpha=0, a=0.5, d=0.2, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.RevoluteDH(alpha=np.pi, a=0.5, d=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
            rtb.PrismaticDH(theta=0, alpha=0, a=0, I=np.array([[0,0,0],[0,0,0],[0,0,1/12]]), m=1),
        ], name="scararob", gravity=[0, 0, 9.81])

    # Get D,C,phi
    D = robot1.inertia([t1,t2,t3])
    C = robot1.coriolis([t1,t2,t3],[dt1,dt2,dt3])
    phi = robot1.gravload([t1,t2,t3])
    phi = np.reshape(phi, (3, 1))

    #diagonal terms D to dkk
    dkk = np.array([[D[0][0], 0, 0],[0, D[1][1],0],[0, 0,D[2][2]]])
    # modelling disturbance, you can more disturbances here
    dk =   np.matmul(C, td) +np.matmul((D-dkk), tdd) +phi
    #motor parameters
    Jm = np.array([[Jm1,0,0],[0,Jm2,0],[0,0,Jm3]])
    Jeff1 = Jm1 + r1 * r1 * dkk[0][0]
    Jeff2 = Jm2 + r2 * r2 * dkk[1][1]
    Jeff3 = Jm3 + r3 * r3 * dkk[2][2]
    Beff1 = Bm1 + Kb1 * Km1 / R1
    Beff2 = Bm2 + Kb2 * Km2 / R2
    Beff3 = Bm3 + Kb3 * Km3 / R2
    K1 = Km1 / R1
    K2 = Km2 / R2
    K3 = Km3 / R3
    #gear ratio will assume same gear ratio so will only use scalar r1
    r = np.array([[r1, 0, 0],[0, r2,0],[0,0,r3]])
    Jeff = np.array([[Jeff1, 0,0],[0, Jeff2,0],[0,0,Jeff3]])
    Jeffinv = np.linalg.inv(Jeff)
    Beff = np.array([[Beff1, 0,0],[0, Beff2,0],[0,0,Beff3]])

    K = np.array([[K1,0,0],[0,K2,0],[0,0,K3]])/dt

    Kinv = np.linalg.inv(K)

    # compute errors
    e = np.array([[qd1-q[0]],[qd2-q[1]],[qd3-q[2]]])
    e = np.reshape(e,(3,1))
    ed = np.array([[0 - q[3]], [0 - q[4]], [0 - q[5]]])
    ed = np.reshape(e, (3, 1))
    
    #PID Gains
    f=np.matmul(np.matmul(Jeff,  Kinv),[ddqd1,ddqd2,ddqd3])+np.matmul(np.matmul(Beff,  Kinv),[dqd1,dqd2,dqd3])
    print(f)

    Kp = wn * wn * np.matmul(Jeff,  Kinv)
    
    Kd = np.matmul((2 * zeta * wn * Jeff - Beff ),Kinv) # change zeta here
    tau=dk

    dt = 0.01
    # if i == 0:
    #     tprev = 0
    #     dt = t - tprev
    # else:
    #     dt = t - tprev
    #disp(size(dt));
    #sum = np.matmul(Ki, e) * dt

    V = np.matmul(Kp , e) + np.matmul(Kd ,ed) + f +tau# voltage control input

    # dynamic equation in joint varaibles
    ddq = np.matmul(np.linalg.inv(Jeff),(r1*np.matmul(K ,V) - (r1**2)*(dk)- np.matmul(Beff , td)))
    # find statedot
    dq = [q[3],q[4],q[5],ddq[0][0],ddq[1][0],ddq[2][0]]

    
    if t==5:
      dq=[100,100,100,10000,10000,10000]

    #print('t= '+str(t))
    # tprev = t used for Integral
    # i = i+1
    return dq


In [ ]:
#Suraj code main
#####################################################################################################################
## DEFINE DH PARAMETERS

# link1
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]
# plot scara
ax = plotrobotscara(DH) # to see plot need to do plt.show()
plt.show()
## OBTAIN JOINT TRAJECTORY USING END EFFECTOR TRAJECTORY
q0 = scara_inverse_kinematics(DH,0.4,0.01,0.1)
q1 = scara_inverse_kinematics(DH,0.4,0.06,0.1)
q1_poly = cubic(q0[0], q1[0], 0, 0, 0, 1)
q2_poly = cubic(q0[1], q1[1], 0, 0, 0, 1)
q3_poly = cubic(q0[2], q1[2], 0, 0, 0, 1)

# time from (t0,tfinal,no)
t=np.linspace(0,1,100)

q = np.transpose(np.zeros(6))
i = 0
alpha1 = 0
d1 = 0.2
a1 = 0.5
# link2
alpha2 = np.deg2rad(180)
d2 = 0
a2 = 0.5
# link3
alpha3 = np.deg2rad(0)
theta3 = 0
a3 = 0
d3 = 0
DH = [[0, alpha1, d1, a1], [0, alpha2, d2, a2], [theta3, alpha3, 0, a3]]  # sequence [theta,alpha,d,a]
# initial configuration
qi = scara_inverse_kinematics(DH,0.4,0.01,0.1)
#start state for integrator [q,dq] last three values indicate joint rate
q0 = [qi[0],qi[1],qi[2],0,0,0]
# motor and link parameter values
m1=1 ; m2=1 ; l1=1;l2=1; l3=1; I1=1/12; I2=1/12; I3=1/12; lc1=l1/2; lc2=l2/2; lc3=l3/2; g=9.81
Jm1 =0.4*10**-4; Jm2 =0.4*10**-4; Jm3=0.4*10**(-4); Bm1 = 4*10**(-5); Bm2 = 4*10**-5; Bm3=4*10**-5; Km1 =2*10**-2; Km2 =2*10**-2;Km3 =2*10**-2
Kb1 = 0.02;Kb2 = 0.02;Kb3=0.02;R1 = 0.3;R2 = 0.3;R3 = 0.3;r1 = 100;r2 = 100; r3=100; zeta = 1; wn = 50; K1 = 1; K2 = 1; K3 = 1

i = 0
### integrate the robot dynamics and control
q = integrate.odeint(modelscara,q0,t,args=(i, I1, I2, l1, l2,l3, lc1, lc2,lc3, m1, m2, Jm1, Jm2, Jm3, r1, r2,r3, R1, R2,R3, Km1, Km2,Km3, Kb1, Kb2, Kb3,Bm1, Bm2, Bm3, zeta, wn,K1,K2,K3,q1_poly,q2_poly,q3_poly))


xed = 0.4 * np.ones(100) # end-effector path only for plotting purpose
yed = np.linspace(0.01, 0.06, 100)
zed = 0.1 * np.ones(100)

DH_curr=[[q[:,0], alpha1, d1, a1], [q[:,1], alpha2, d2, a2], [theta3, alpha3, q[:,2], a3]]
xe,ye,ze=forward_kinematics2(DH_curr)


plt.figure(1)
tt = np.linspace(0,1,100)


# generate actual vs desired plots here

f=plt.figure(2)
for i in range(len(tt)):
  ax=f.add_subplot(111)
  ax.plot(tt,xed,color='r',label='Desired trajectory x')
  ax.plot(tt,xe,color='g',label='Actual trajectory x')

f1=plt.figure(3)
for i in range(len(tt)):
  ax=f1.add_subplot(111)
  ax.plot(tt,yed,color='r',label='Desired trajectory y')
  ax.plot(tt,ye,color='g',label='Actual trajectory y')
  
# Animate in 3D
for i in range(len(tt)):
    # substitute joint variables obtained from integration
    DH = [[q[i][0], alpha1, d1, a1], [q[i][1], alpha2, d2, a2], [theta3, alpha3, q[i][2], a3]]
    ax = plotrobotscara(DH)
    ax.plot3D(xed, yed, zed, '-m', label=r'$desired end-effector trajectory$')
    plt.draw()
    ax.text(0.5, 0.5, 0, 'step no ='+str(i), color='red')
    plt.pause(0.001)

